In [1]:
%matplotlib widget
%reload_ext autoreload
from Observation import *
from scipy.optimize import curve_fit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import re
import glob, os
from astropy import units as u
import math as mt
import cmocean
from matplotlib.colors import LogNorm
from scipy.stats import norm
from matplotlib.offsetbox import AnchoredText
from astropy.cosmology import Planck18 as cosmo

sheet_id = "1Ox0uxEm2TfgzYA6ivkTpU4xrmN5vO5kmnUPdCSt73uU"
sheet_name = "instruments.csv"
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"

try:
    instruments = Table.from_pandas(pd.read_csv(url))
except Exception:
    instruments = Table.read("Instruments.csv")
instruments = instruments[instruments.colnames]
instruments_dict ={ name:{key:float(val) for key, val in zip(instruments["Charact."][:],instruments[name][:])} for name in instruments.colnames[3:]}




def dark_plateau(T=-80,a=0.23362821,plateau=0.44384484):
    T = T+273.15
    E = 1.1557 - (7.021*1e-4*T**2)/(1108+T)
    k = 8.62e-5
    return 2.55*1e15*a*3600*0.0013**2*T**(3/2)*np.exp(-E/(2*k*T))+plateau



def mostFrequent(arr):
    n=len(arr)
    # Insert all elements in Hash.
    Hash = dict()
    for i in range(n):
        if arr[i] in Hash.keys():
            Hash[arr[i]] += 1
        else:
            Hash[arr[i]] = 1
    # find the max frequency
    max_count = 0
    res = -1
    for i in Hash:
        if (max_count < Hash[i]):
            res = i
            max_count = Hash[i]
         
    return res

gaus = lambda x, a, xo, sigma, offset: a ** 2 * np.exp(-np.square((x - xo) / sigma) / 2)+ offset

n2,n1=100,500


class ExposureTimeCalulator(widgets.HBox):
    @initializer
    def __init__(self, instrument="FIREBall-2 2023",x_axis='exposure_time', time_max = 2005, **kwargs):#, Atmosphere=0.5, Throughput=0.13*0.9, follow_temp=False, acquisition_time=1, Sky=4, Signal=24, EM_gain=1400,RN=109,CIC_charge=0.005, Dard_current=0.08,readout_time=1.5,counting_mode=False,smearing=0.7,extra_background=0,temperature=-100,PSF_RMS_mask=2.5,PSF_RMS_det = 3.5,QE=0.45,cosmic_ray_loss_per_sec=0.005,
        """
        Generate an ETC app containing multiple widghet that allow to change the ETC parameters
        as well as plotting the result (e- and noise budget, limiting flux, SNR) in terms of the different parameters.
        """
        super().__init__()
        args, _, _, locals_ = inspect.getargvalues(inspect.currentframe())
        for i in range(len(instruments)):
            setattr(self, instruments["Charact."][i], float(instruments[instrument][i]) )
        exposure_time=np.logspace(0,np.log10(time_max))

        self.output = widgets.Output()
        time=exposure_time
        self.time =  float(instruments[instrument][instruments["Charact."]=="exposure_time"][0])
        i = np.argmin(abs(self.time - exposure_time))
        self.arg=i
        # print(self.arg,i,time[i],time[i],self.time)
        IFS = False if self.dimensions==2 else True

        self.new = Observation(instrument=instrument,smearing=self.smearing,counting_mode=False,exposure_time=exposure_time,Sky=self.Sky,acquisition_time=self.acquisition_time,Signal=self.Signal,EM_gain=self.EM_gain,RN=self.RN, CIC_charge=self.CIC_charge, Dard_current=self.Dard_current,PSF_RMS_mask=self.PSF_RMS_mask,PSF_RMS_det=self.PSF_RMS_det,QE=self.QE, extra_background=self.extra_background,
            Collecting_area=self.Collecting_area, pixel_scale=self.pixel_scale, Throughput=self.Throughput, Spectral_resolution=self.Spectral_resolution, Slitwidth=self.Slitwidth, dispersion=self.dispersion,
            PSF_source=self.PSF_source,Line_width=self.Line_width,wavelength=self.wavelength, Atmosphere=self.Atmosphere, pixel_size=self.pixel_size,cosmic_ray_loss_per_sec=self.cosmic_ray_loss_per_sec,readout_time=self.readout_time, Slitlength=self.Slitlength,i=self.arg,Δλ=0,Δx=0,IFS=IFS)


        self.x = exposure_time
        
        
        style={}
        width = '400px'
        width = '500px'
        small = '247px'
        small = '230px'
        psmall = '186px'
        vsmall = '147px'
        c_update=False
        # ALL THE DIFFERENT VARIATION WITH TEMPERATURE
        self.smearing_poly = np.poly1d([-0.0306087, -2.2226087])#np.poly1d([-0.0453913, -3.5573913])
        # self.dark_poly = np.poly1d([2.13640462e-05, 7.83596239e-03, 9.57682651e-01, 3.86154296e+01])#with plateau
        # self.dark_poly = np.poly1d([0.07127906, 6.83562573]) #without plateau# does to low because only calibrated down to -100 but then it kinda saturated. maybe because of CIC?
        self.dark_poly = dark_plateau
        # self.CIC_poly = np.poly1d([1.57925408e-05, 2.80396270e-03, 1.34276224e-01]) #without plateau# does to low because only calibrated down to -100 but then it kinda saturated. maybe because of CIC?


        # self.other_options =  ['exposure_time','acquisition_time',"Signal","RN","Dard_current" ,'Sky',"readout_time","PSF_RMS_det","QE","cosmic_ray_loss_per_sec","Throughput","Atmosphere","lambda_stack","PSF_source",'Collecting_area',"Δx","Δλ"]#[:-3]
        self.other_options =  [
                               "------DECTECTOR PERFORMANCE","QE","RN","Dard_current" ,"cosmic_ray_loss_per_sec", #,"pixel_size"
                               "-------------OBSERVED SOURCE","Signal",'Sky',"PSF_source","Line_width",
                               "--------OBSERVATION STRATEGY", "Atmosphere",'exposure_time','acquisition_time',"readout_time","lambda_stack",  "wavelength", #,"Δx","Δλ"
                               "-----------INSTRUMENT DESIGN",'Collecting_area',"pixel_scale","Throughput","PSF_RMS_mask","PSF_RMS_det",
                               "---------SPECTROGRAPH DESIGN","Spectral_resolution","Slitwidth","Slitlength","dispersion",
        # "---FIREBall specific---",
        ]#[:-3]
        self.fb_options_no_temp = self.other_options + ["----------AMPLIFIED DECTECTOR","EM_gain","CIC_charge","smearing","extra_background"]
        self.fb_options = self.fb_options_no_temp + ["temperature"]

        self.instrument = widgets.Dropdown(options=instruments.colnames[3:],value=instrument,description="Instrument", layout=Layout(width=small),description_tooltip="Instrument characteristics",continuous_update=c_update)
        self.ylog = widgets.Checkbox(value=False,description='ylog',disabled=False, layout=Layout(width='147px'),description_tooltip="Use this check box to use a log scale for the y axis",continuous_update=c_update)
        self.xlog = widgets.Checkbox(value=True,description='xlog',disabled=False, layout=Layout(width='147px'),description_tooltip="Use this check box to use a log scale for the x axis",continuous_update=c_update)
        self.IFS = widgets.Checkbox(value=IFS,description='IFS',disabled=False, layout=Layout(width='147px'),description_tooltip="Check this box if the instrument is an integral field spectrograph (not just a single slit of fiber)",continuous_update=True)
       
        self.Signal = widgets.FloatLogSlider(min=-21, max=-9 ,value=self.Signal,description='Source brightness', style =dict(description_width='initial'), layout=Layout(width=width),description_tooltip="Flux of the diffuse source in ergs/cm2/s/arcsec2/Å.",continuous_update=c_update)
        self.Sky = widgets.FloatLogSlider( min=-23, max=-15,value=self.Sky,base=10, style =dict(description_width='initial'), layout=Layout(width=width),description='Sky    brightness',description_tooltip="Level of sky background illumination (zodiacal and galactic) in ergs/cm2/s/arcsec2/Å ",continuous_update=c_update)
        self.acquisition_time = widgets.FloatLogSlider( min=-1, max=3,value=self.acquisition_time,style =style ,base=10,layout=Layout(width=width),description='Taq (h)',description_tooltip="Total acquisition time [hours]",continuous_update=c_update)
        self.exposure = widgets.FloatRangeSlider( min=0, max=time_max,value=(self.readout_time,self.time),style = style, layout=Layout(width=width),description='Rd/Exp time',step=0.1,readout_format='.0f',description_tooltip="Readout time and exposure time [seconds]",continuous_update=c_update)

        self.fwhm = widgets.FloatRangeSlider( min=0.01, max=6,value=(self.PSF_RMS_mask,self.PSF_RMS_det),style = style, layout=Layout(width=width),description='Mask/det σ',step=0.01,readout_format='.2f',todescription_tooltipoltip="Spatial resolution in arcseconds respectively at the mask and detector level. To be multiplied by 2.35 to have the FWHM.",continuous_update=c_update)
        self.RN = widgets.FloatSlider( min=0.01, max=120,value=self.RN, style = style, step=0.1, layout=Layout(width=width),description='Read noise',description_tooltip="Detector readout noise in electrons/pixel",continuous_update=c_update)
        self.QE = widgets.FloatSlider( min=0.01, max=1,value=self.QE,style = style, layout=Layout(width=width),description='QE',step=0.01,readout_format='.2f',description_tooltip="Detector quantum efficiency",continuous_update=c_update)
        self.Dard_current = widgets.FloatSlider( min=0, max=50,value=self.Dard_current, style = style, layout=Layout(width=width),description='Dard current',step=0.0011,readout_format='.2f',description_tooltip="Detector dark current [e-/pix/hour]",continuous_update=c_update)

        self.extra_background = widgets.FloatSlider( min=0, max=20,value=self.extra_background,style = style, layout=Layout(width=width),description='Extra bckgnd',step=0.2,readout_format='.1f',description_tooltip="Additional background on the detector [e-/pix/hour]",continuous_update=c_update)
        self.EM_gain = widgets.IntSlider( min=1, max=3500,value=self.EM_gain, style = style, layout=Layout(width=width),description='EM gain',description_tooltip="EMCCD amplification gain in e-/e-",continuous_update=c_update)
        self.CIC_charge = widgets.FloatSlider( min=0, max=0.07,value=self.CIC_charge,style = style, layout=Layout(width=width),description='CIC charge',step=0.001,readout_format='.3f',description_tooltip="EMCCD spurious charges due to amplification in electrons [e-/pix]",continuous_update=c_update)
        self.follow_temp = widgets.Checkbox(value=False,description='Temp',disabled=False, layout=Layout(width=vsmall),description_tooltip="Check this box to force charge transfer efficiency and dark current levels to be fixed by the temperature widget. Interesting feature to optimize EMCCD temperature.",continuous_update=c_update)
        self.counting_mode = widgets.Checkbox(value=False,description='γ-Threshold',disabled=False, layout=Layout(width=psmall),description_tooltip="Check this box to apply thresholding photon counting processing. The efficiency of this process is determined by the gain, read noise, smearing, flux.",continuous_update=c_update)
        self.temperature = widgets.FloatSlider( min=-120, max=-85,value=-115, style = style,description=r'Temp (C)',step=0.1, layout=Layout(width=width),description_tooltip="EMCCD's Temperature in Celcius degrees: determines its charge transfer efficiency and dark current rate.",continuous_update=c_update)
        self.smearing = widgets.FloatSlider( min=0, max=self.smearing_poly(-120),value=self.smearing, layout=Layout(width=width),description='Smearing',step=0.01,description_tooltip="Smearing length of the EMCCD (exponential length in pixels). This length, representing the charge transfer efficiency is fixed by the temperature when the Temp checkbox is checked.",continuous_update=c_update)
       
        self.Collecting_area = widgets.FloatLogSlider( min=-2, max=3,value=np.log10(self.Collecting_area), style =style,base=10, layout=Layout(width=width),description='Area',description_tooltip="Collecting area of the instrument in square meter",continuous_update=c_update)
        self.pixel_scale = widgets.FloatSlider( min=0.01, max=50,value=self.pixel_scale,base=10, style =style, layout=Layout(width=width),description='Pixel scale',description_tooltip="Pixel plate scale in  ''/pix",continuous_update=c_update)
        self.Throughput = widgets.FloatSlider( min=0.01, max=1,value=self.Throughput,base=10, style =style, layout=Layout(width=width),description='Throughput',description_tooltip="Instrument throughput at effective wavelength (not accounting for detector quantum efficiency and atmospheric transmission)",continuous_update=c_update)


       
        self.Atmosphere = widgets.FloatSlider( min=0.1, max=1,value=self.Atmosphere,base=10, style =style, layout=Layout(width=width),description='Atmosphere',description_tooltip="Atmospheric transmission",continuous_update=c_update)
        self.pixel_size = widgets.FloatSlider( min=2, max=40,value=self.pixel_size,base=10, style =style, layout=Layout(width=width),description='Pix size',description_tooltip="Pixel size in microns",continuous_update=c_update)

        self.PSF_source = widgets.FloatSlider( min=0.01, max=100,value=self.PSF_source,base=10, style =style, layout=Layout(width=width),description='σ Source',description_tooltip="Spatial extension of the source in arcseconds",continuous_update=c_update)
        self.Line_width = widgets.FloatSlider( min=0.01, max=100,value=self.Line_width,base=10, style =style, layout=Layout(width=width),description='Eq width (Å)',description_tooltip="Spectral extension of the source/emission line in Å ",continuous_update=c_update)


        self.wavelength = widgets.FloatSlider( min=50, max=1000,value=self.wavelength,base=10, style =style, step=0.1, layout=Layout(width=width),description='Observed λ',description_tooltip="Oberved λ in Å (only used for conversions)",continuous_update=c_update)
        self.Δλ = widgets.FloatSlider( min=-150, max=150,value=0,base=10, style =style, layout=Layout(width='400px'),description='Δλ',description_tooltip="Distance to the emission line being analyzed [Å]",continuous_update=c_update)
        self.Δx = widgets.FloatSlider( min=-15, max=15,value=0,base=10, style =style, layout=Layout(width='400px'),description='Δx',description_tooltip="Distance to the source being analyzed ['']",continuous_update=c_update)
        self.Throughput_FWHM = widgets.FloatLogSlider( min=1, max=5,value=np.log10(self.Throughput_FWHM),base=10, style =style, layout=Layout(width='400px'),description='TH FWHM',description_tooltip="Instrument throughput FWHM in  Å")#5.57e-18
        self.redshift = widgets.FloatSlider( min=0, max=13,value=0,base=10, style =style, layout=Layout(width='400px'),description='Redshift',description_tooltip="Redshift only considered to shift blackbody spectra or QSO/Star spectra. Flux will stay based on source observed surface brightness. Changes also the kpc size",continuous_update=c_update)
        self.lambda_stack = widgets.IntSlider( min=1, max=200,value=1, layout=Layout(width=width),description='λ Stack (pix)',step=0.1,description_tooltip="Number of spectral slices used to stack cube (spectral pix)",continuous_update=c_update)

        self.Spectral_resolution = widgets.IntSlider( min=90, max=10000,value=self.Spectral_resolution,base=10, style =style, layout=Layout(width=width),description='R (λ/dλ) ',description_tooltip="Instrument spectral resolution λ/dλ",continuous_update=c_update)
        # self.Slitwidth = widgets.FloatSlider( min=0.1, max=600,value=self.Slitwidth,base=10, style =style, layout=Layout(width=width),description='Slit ["]',description_tooltip="Width of the slit [''] ")
        self.SlitDims = widgets.FloatRangeSlider( min=0.1, max=600,value=(self.Slitwidth,self.Slitlength),base=10, style =style, layout=Layout(width=width),description='Slit dims["]',description_tooltip="Width and length of the slit [''] (put same value for fibers) ")
        
        
        self.dispersion = widgets.FloatSlider( min=0.01, max=15,value=self.dispersion,base=10, style =style, step=0.001, readout_format='.2f', layout=Layout(width=width),description='Dispersion',description_tooltip="Dispersion at the detector Å/pix ",continuous_update=c_update)

        self.cosmic_ray_loss_per_sec = widgets.FloatSlider( min=0, max=0.1,value=self.cosmic_ray_loss_per_sec,base=10, style =style,step=0.0001,readout_format='.4f', layout=Layout(width=width),description='CR loss',description_tooltip="Cosmic ray loss per second. eg. 0.01 would mean that 1 sec image looses 1% pixels due to cosmic rays",continuous_update=c_update)

        gals = ["Rest-frame: COSMOS " + os.path.basename(f).replace(".txt","") for f in glob.glob("Spectra/GAL_COSMOS_SED/*.txt")]
        QSOs = ["Rest-frame: Salvato " + os.path.basename(f).replace(".txt","") for f in glob.glob("Spectra/QSO_SALVATO2015/*.txt")]
        self.spectra     = widgets.Dropdown(options=["Observed-frame: Baseline Spectra"]  + gals + QSOs + ["Rest-frame: Blackbody 5900 K (09V)","Rest-frame: Blackbody 1500 K (BOV)","Rest-frame: Blackbody 9000 K (B3V)","Rest-frame: Blackbody 480 K (AOV)","Rest-frame: Blackbody 8810 K (A2V)","Rest-frame: Blackbody 8160 K (A5V)","Rest-frame: Blackbody 7020 K (FOV)","Rest-frame: Blackbody 6750 K (F2V)","Rest-frame: Blackbody 6530 K (F5V)","Rest-frame: Blackbody 930 K (GOV)","Rest-frame: Blackbody 5830 K (G2V)","Rest-frame: Blackbody 5560 K (G5V)","Rest-frame: Blackbody 240 K (KOV)","Rest-frame: Blackbody 5010 K (K2V)","Rest-frame: Blackbody 4560 K (K4V)","Rest-frame: Blackbody 4340 K (K5V)","Rest-frame: Blackbody 4040 K (K7V)","Rest-frame: Blackbody 3800 K (MOV)","Rest-frame: Blackbody 3530 K (M2V)","Rest-frame: Blackbody 3380 K (M3V)","Rest-frame: Blackbody 3180 K (M4V)","Rest-frame: Blackbody 3030 K (M5V)","Rest-frame: Blackbody 2850 K (M6V)"] + ["Observed-frame: UVSpectra 1538p477 NUV~16.6","Observed-frame: UVSpectra 1821p643 NUV~14",'Observed-frame: UVSpectra 0044p030 NUV~16.5',"Observed-frame: UVSpectra mrk509","Observed-frame: UVSpectra 2344p092","Observed-frame: UVSpectra 1637p574","Observed-frame: UVSpectra 1115p080","Observed-frame: UVSpectra 0414m060","Observed-frame: UVSpectra 0115p027","Observed-frame: UVSpectra 2251p113","Observed-frame: UVSpectra 2201p315","Observed-frame: UVSpectra 1928p738","Observed-frame: UVSpectra 1700p518"], layout=Layout(width='350px'),description='Entry',value="Observed-frame: Baseline Spectra",continuous_update=c_update)
        self.units       = widgets.Dropdown(options=["ADU/frame","amplified e-/frame","e-/frame","photons/frame","e-/hour","amplified e-/hour","photons/hour"], layout=Layout(width='350px'),description='Units',value="ADU/frame")# TODO add ergs/cm2/...
        # widgets.Dropdown(options=["S2: 0.053 ADU/e-, FW=5.6 KADU","S2_hdr: 0.97 ADU/e-, FW=52 KADU","1: 0.02 ADU/e-, FW=2.1 KADU","1': 0.4 ADU/e-, FW=40 KADU","2: 0.04 ADU/e-, FW=4.7 KADU","2018: 0.5 ADU/e-, FW=56 KADU","2022: 0.2 ADU/e-, FW=22 KADU","2023_noOS: 0.04 ADU/e-, FW=39 KADU"], layout=Layout(width='350px'),description='RO seq',value="S2_hdr: 0.97 ADU/e-, FW=52 KADU",continuous_update=c_update)
        self.QElambda = widgets.Checkbox(value=True,description='QE(λ)',disabled=False,tooltip="Check this box to apply λ QE dependancy",layout=Layout(width="217px"))
        self.atmlambda = widgets.Checkbox(value=True,description='atm(λ)',disabled=False,tooltip="Check this box to apply λ atm transmission dependancy",layout=Layout(width="217px"))
        self.sky_lines = widgets.Checkbox(value=True,description='Sky lines',disabled=False,tooltip="Check this box to add sky emission lines",layout=Layout(width="217px"))

        self.fraction_lya = widgets.FloatSlider( min=0, max=0.2,value=0.05,style = style, layout=Layout(width='217px'),description='Lya fraction',step=0.001,readout_format='.2f',tooltip="Fraction of E(Lya)/E(NUV)")
        self.fraction_lya.layout.visibility = 'hidden'  

        self.save_plot_button = widgets.Button(description="Save Plot", layout=Layout(width='auto'))
        self.save_data_button = widgets.Button(description="Save Data", layout=Layout(width='auto'))


        self.reset = widgets.Button(value=False,description='↺',disabled=False,button_style='', layout=Layout(width="30px")) 


        self.change = widgets.Checkbox(value=True,description='change',disabled=False, layout=Layout(width='147px'),description_tooltip="Use this check box to use a log scale for the x axis")
        self.change.layout.visibility = 'hidden'  

        # self.lambda_stack = widgets.FloatSlider( min=10*self.wavelength/self.Spectral_resolution, max=self.Bandwidth*10,value=self.Bandwidth, layout=Layout(width=width),description='λ width [Å]',step=0.1,description_tooltip="Wavelength range used to stack signal. Min = 1 resolution element = 5pixels = 1Å, Max = total spectra =bandwidth = 200Å")   
       
        if "FIREBall" in instrument:
            options = self.fb_options if self.follow_temp.value else self.fb_options_no_temp
        else:
            options = self.other_options
        self.x_axis=widgets.Dropdown(options=options,value=self.x_axis,description='X axis', layout=Layout(width=small),description_tooltip="Variable used to analyze the evolution of the SNR.")

        if "FIREBall" not in instrument:
            self.counting_mode.layout.visibility = 'hidden'  
            self.follow_temp.layout.visibility = 'hidden'  

        self.smearing.layout.visibility = 'hidden'
        self.temperature.layout.visibility = 'hidden'
        if self.follow_temp.value:
            self.Dard_current.value = self.dark_poly(self.temperature.value)#10**
            self.smearing.value = self.smearing_poly(self.temperature.value)
            # self.CIC_charge.value = self.CIC_poly(self.temperature.value)
  

        # print(self.new.QE,self.new.N_images_true)
        # self.Signal_el = self.new.Signal_el

        wids = widgets.interactive(self.update,x_axis=self.x_axis,xlog=self.xlog,log=self.ylog, smearing=self.smearing,counting_mode=self.counting_mode,exposure=self.exposure,Sky=self.Sky,acquisition_time=self.acquisition_time,Signal=self.Signal,EM_gain=self.EM_gain,RN=self.RN, CIC_charge=self.CIC_charge, Dard_current=self.Dard_current,temperature=self.temperature,follow_temp=self.follow_temp,fwhm = self.fwhm,QE=self.QE, extra_background=self.extra_background,
            Collecting_area=self.Collecting_area, pixel_scale=self.pixel_scale, Throughput=self.Throughput, Spectral_resolution=self.Spectral_resolution, SlitDims=self.SlitDims, dispersion=self.dispersion,
            PSF_source=self.PSF_source,Line_width=self.Line_width,wavelength=self.wavelength,Δλ=self.Δλ,Δx=self.Δx, Atmosphere=self.Atmosphere, pixel_size=self.pixel_size,cosmic_ray_loss_per_sec=self.cosmic_ray_loss_per_sec,lambda_stack=self.lambda_stack,change=self.change,
            spectra=self.spectra,units=self.units,Throughput_FWHM=self.Throughput_FWHM, QElambda=self.QElambda, atmlambda=self.atmlambda, fraction_lya=self.fraction_lya,sky_lines=self.sky_lines,redshift=self.redshift, IFS=self.IFS)
        
        wids2 = widgets.interactive(self.update_instrument,instrument=self.instrument)
        # Question: why do we need to reset it to True here?
        self.change.value=True

        def reset(_):
            self.update_instrument(self.instrument.value)
            self.Signal.value = instruments_dict[self.instrument.value]["Signal"]
        self.reset.on_click(reset)

        def save_plot(_):
            self.fig.savefig("/tmp/fig1.png")
            self.fig2.savefig("/tmp/fig2.png")
            self.fig3.savefig("/tmp/fig3.png")
        self.save_plot_button.on_click(save_plot)

        def save_data(_):
            self.f = lambda x: self.wavelength.value + (self.dispersion.value/10) * (x - n1/2)
            fitswrite(self.ifs_cube.T,"/tmp/ifs_cube.fits")
            fitswrite(np.transpose(self.ifs_cube_stack,(1,2,0)),"/tmp/ifs_cube_stack.fits")
            fitswrite(self.imaADU_without_source,"/tmp/imaADU_without_source.fits")
            fitswrite(self.imaADU_source,"/tmp/imaADU_source.fits")
            fitswrite(self.imaADU_stack_without_source,"/tmp/imaADU_stack_without_source.fits")
            fitswrite(self.imaADU_stack_only_source,"/tmp/imaADU_stack_only_source.fits")
            np.savetxt("/tmp/spectra.csv", np.asarray([ self.f(np.arange(n1)), self.ifs_spectra[0].get_ydata(), self.ifs_spectra_stack[0].get_ydata(), self.ifs_spectra_background[0].get_ydata(), self.ifs_spectra_background_stack[0].get_ydata()  ]), delimiter=",",header="wavelength,ifs_spectra, ifs_spectra_stack, ifs_spectra_background, ifs_spectra_background_stack")
            return
        self.save_data_button.on_click(save_data)


        self.obs_tab = VBox([HBox([self.Signal,self.PSF_source ]), HBox([self.Sky,self.Line_width])]) 
        self.strat_tab = VBox([HBox([self.Atmosphere,self.acquisition_time ]), HBox([self.exposure,self.wavelength])  , HBox([self.lambda_stack])     ])
        self.inst_tab = VBox([HBox([self.Collecting_area,self.pixel_scale ]), HBox([self.Throughput,self.fwhm ])])
        self.spectro_tab = VBox([HBox([self.Spectral_resolution,self.SlitDims ]), HBox([self.dispersion,self.IFS ])]) 
        self.det_tab = VBox([HBox([self.QE,self.RN ]), HBox([self.Dard_current,self.pixel_size]),          HBox([self.cosmic_ray_loss_per_sec,self.extra_background ]), HBox([self.EM_gain,self.CIC_charge])])
        # self.amp_tab = HBox([self.EM_gain,self.CIC_charge])
        self.fb_tab = VBox([HBox([self.counting_mode,self.follow_temp ]),HBox([self.temperature,self.smearing]),HBox([self.change]) ])
        self.im_tab =         VBox([HBox([self.spectra,self.redshift,self.Throughput_FWHM ]), HBox([self.units,self.Δλ,self.Δx])   , HBox([self.QElambda,self.atmlambda,self.sky_lines])  ]) #self.fraction_lya
        
        if ("FIREBall-2" in instrument):
            tab_contents = [ "Observed Source", "Observation strategy" , "Instrument Design",  "Spectrograph Design","Detector performance","Imaging"]#, 'FIREBall specific'] #,"Amplified Detector"
            children = [ self.obs_tab, self.strat_tab,  self.inst_tab, self.spectro_tab, self.det_tab, self.im_tab]#, self.fb_tab]
        else:
            tab_contents = [ "Observed Source", "Observation strategy" , "Instrument Design",  "Spectrograph Design","Detector performance"]#,"Imaging"] #,"Amplified Detector"
            children = [ self.obs_tab, self.strat_tab,  self.inst_tab, self.spectro_tab, self.det_tab]#, self.im_tab] 




         #HBox([self.instrument,self.x_axis,self.xlog, self.ylog ]),    
         # HBox([ self.counting_mode,self.follow_temp,self.extra_background ,self.temperature,self.smearing])     ]
        self.controls = widgets.Tab()# Accordion
        self.controls.children = children
        for i, name in enumerate(tab_contents):
            self.controls.set_title(i, name)

        self.out1 = widgets.Output()
        self.out2 = widgets.Output()
        self.out3 = widgets.Output()
        # self.output_tabs = widgets.Tab(children = [self.out2, self.out1]);self.output_tabs.set_title(0, 'Image');self.output_tabs.set_title(1, 'SNR')
        if self.IFS.value is False:
            self.output_tabs = widgets.Tab(children = [self.out1, self.out2]); self.output_tabs.set_title(0, 'SNR');self.output_tabs.set_title(1, 'Spectral image')
            #;self.output_tabs.set_title(2, 'IFS Image')
            self.plot_shown = False#  # Reset the plot flag when hiding tab 3    # self.output_tabs.selected_index = 0  # Sélectionner un autre onglet par défaut
            self.first_plot = True
        else:    
            self.output_tabs = widgets.Tab(children = [self.out1, self.out2, self.out3]); self.output_tabs.set_title(0, 'SNR');self.output_tabs.set_title(1, 'Spectral image');self.output_tabs.set_title(2, 'IFS Image')
            self.plot_shown = True#  # Reset the plot flag when hiding tab 3    # self.output_tabs.selected_index = 0  # Sélectionner un autre onglet par défaut
            self.first_plot = False

        def on_tab_change(change): # TODO be sure that this works in all the cases
            self.on_instrument_change()



        self.output_tabs.observe(on_tab_change, names='selected_index')
        
        
        
        new = VBox([ HBox([self.instrument,self.x_axis,self.xlog, self.ylog,self.reset,self.save_plot_button,self.save_data_button ])  ,self.controls , self.output_tabs   ]) #


        with self.out1: # before because then the arrays are transformed into numbers
            # print(self.IFS.value)
            self.fig = self.new.PlotNoise(x=x_axis)
            args, _, _, locals_ = inspect.getargvalues(inspect.currentframe())
            self.v=[]
            for j, ax in enumerate(self.fig.axes):
                if j==2:
                    label = '%s [Best]=%s [%s]\nSNR [Best]=%0.2f [%0.2f]'%(x_axis,float_to_latex(self.time),float_to_latex(exposure_time[np.nanargmax(self.new.SNR)]),self.new.SNR[self.arg],np.nanmax(self.new.SNR))#, self.new.gain_thresholding[arg])
                    self.v.append(ax.axvline(self.time,ls=':',c='k',label=label))
                    ax.legend(loc='upper right')
                else:
                    self.v.append(ax.axvline(self.time,ls=':',c='k'))
            self.ax0 =  self.fig.axes[0]
            self.ax1 =  self.fig.axes[1]
            self.ax2 =  self.fig.axes[2]
            self.ax3 =  self.fig.axes[3]
            self.ax3.set_ylim((-19,-12.5))
            self.ax0.set_xscale('log')
            self.fig.canvas.toolbar_position = 'bottom'
            title = 'Instrument=%s, FOV=%samin$^2$, λ=%inm, Throughput=%i%%, Atm=%i%%, Platescale=%.1f, area=%0.1fm$^2$'%(instrument, instruments[instrument][instruments["Charact."]=="FOV_size"][0], instruments[instrument][instruments["Charact."]=="wavelength"][0], 100*instruments[instrument][instruments["Charact."]=="Throughput"][0], 100*instruments[instrument][instruments["Charact."]=="Atmosphere"][0], instruments[instrument][instruments["Charact."]=="pixel_scale"][0], instruments[instrument][instruments["Charact."]=="Collecting_area"][0])
            self.ax0.set_title(title,y=0.97,fontsize=10)

            plt.show(self.fig)  

        self.im,self.im_stack, self.cube_stack, self.im0, source_im_wo_atm, self.imaADU_stack_only_source, self.imaADU_without_source, self.imaADU_stack_without_source, self.imaADU_source = self.new.SimulateFIREBallemCCDImage( Bias="Auto",  p_sCIC=0,  SmearExpDecrement=50000,  source="Baseline Spectra",size=[n1, n2], OSregions=[0, max(n2,n1)], name="Auto", spectra="-", cube="-", n_registers=604, save=False, field="targets_F2.csv",QElambda=self.QElambda.value,atmlambda=self.QElambda.value,fraction_lya=self.fraction_lya.value, Full_well=self.Full_well, conversion_gain=self.conversion_gain, Throughput_FWHM=self.Throughput_FWHM.value,sky_lines=self.sky_lines.value, redshift=self.redshift.value)
        
        center = n1/2
        f = lambda x: self.wavelength.value + self.dispersion.value * (x - center)
        g = lambda x: (x - self.wavelength.value) / self.dispersion.value + center
     
        with self.out2:
            self.current_cmap = cmocean.cm.deep# current_cmap = cmocean.cm.solar# self.current_cmap = cmocean.cm.thermal
            

            self.current_cmap.set_bad(color='black')
            self.bins=np.arange(-100,4000,100)
            self.bins=np.linspace(-100,np.nanmax(self.im),100)

            self.mod = mostFrequent(self.im_stack[:20,:].flatten())

            self.limit = self.mod+self.new.n_threshold * self.RN.value

            self.fig2 = plt.figure(figsize=(12, 8))
            gs = self.fig2.add_gridspec(2, 2, height_ratios=[0.5, 1])
            self.nax = self.fig2.add_subplot(gs[0, 0])      # Top-left
            self.nax0 = self.fig2.add_subplot(gs[0, 1])     # Top-right
            gs_nax2 = gs[1, 0].subgridspec(2, 1, height_ratios=[1, 1])
            self.nax2_1 = self.fig2.add_subplot(gs_nax2[0, 0])  # Bottom-left, top half
            self.nax2 = self.fig2.add_subplot(gs_nax2[1, 0])    # Bottom-left, bottom half
            self.nax1 = self.fig2.add_subplot(gs[1, 1]) 
            
            self.nax1_secondary = self.nax1.secondary_xaxis("top", functions=(f,g))
            self.nax2_1_secondary = self.nax2_1.secondary_xaxis("top", functions=(f,g))
            self.nax2_1.set_title('Wavelength (nm)',fontsize=10)

            if self.counting_mode.value:
                stacked_image = np.nansum(self.cube_stack>self.limit,axis=0)
                im0 = self.nax0.imshow(stacked_image, aspect="auto",cmap=self.current_cmap)
            else:
                im0 = self.nax0.imshow(self.im_stack, aspect="auto",cmap=self.current_cmap)
            im = self.nax.imshow(self.im, aspect="auto",cmap=self.current_cmap)#, norm=LogNorm())
            labels =  ['%s: %0.3f (%0.1f%%)'%(name,getattr(self.new,"electrons_per_pix")[self.new.i,j],100*getattr(self.new,"electrons_per_pix")[self.new.i,j]/np.sum(getattr(self.new,'electrons_per_pix')[self.new.i,:])) for j,name in enumerate(self.new.names)]
            self.nax.plot(0,0,".",label="\n".join(labels))
            self.nax.legend(loc="upper left",handlelength=0, handletextpad=0, fancybox=True,markerscale=0,fontsize=6)
            # self.nax.text(0,0,"\n".join(labels))
            im2=self.nax0.imshow(self.im_stack, aspect="auto",cmap=self.current_cmap)#, norm=LogNorm())
            self.nax0.get_xaxis().set_ticks([])
            self.nax0.get_yaxis().set_ticks([])
            self.nax.get_xaxis().set_ticks([])
            self.nax.get_yaxis().set_ticks([])
            self.nax.set_title('Single image: FOV = %i" × %iÅ, λ~%iÅ'%(100*self.pixel_scale.value,500*self.dispersion.value,10*self.wavelength.value))
            self.nax0.set_title('Stacked image: Pixel size = %0.2f" × %0.2fÅ'%(self.pixel_scale.value,self.dispersion.value))
            self.nax1.set_title('Wavelength (nm)',fontsize=10)
            # self.nax2.set_title('Histogram')
            # self.nax2.set_ylabel('Occurence')
            self.nax2.set_xlabel('ADU values')
            self.nax1.set_xlabel('Pixels')
            # FIXME besure to only use correctly the factor 2.35
            self.l1 = self.nax1.plot(self.im[:,int(n1/2-self.Line_width.value/self.pixel_scale.value):int(n1/2+self.Line_width.value/self.pixel_scale.value)].mean(axis=1),ls='-',lw=3,label='Single exp profiles',alpha=0.2)
            self.l2 = self.nax1.plot(self.im[int(n2/2 - self.PSF_source.value/self.pixel_scale.value/2/2.35):int(n2/2 + self.PSF_source.value/self.pixel_scale.value/2/2.35),:].mean(axis=0),ls='-',lw=3,alpha=0.2,c="k")



            # self.final_sky_before_convolution =  self.nax2_1.plot(self.new.final_sky_before_convolution*np.ones(n1),ls='-',c="k",label="final_sky_before_convolution")#,lw=3,alpha=0.2)
            # self.atm_trans_before_convolution =  self.nax2_1.plot(self.new.atm_trans_before_convolution*np.ones(n1),ls=':',c="k",label="atm_trans_before_convolution")#,lw=3,alpha=0.2)

            self.absorption         =  self.nax2_1.plot(self.new.atm_trans*np.ones(n1),ls=':',c="k",label=r"Atm$_{Trans}$(λ) for Signal")#,lw=3,alpha=0.2)
            self.emission_lines     =  self.nax2_1.plot(self.new.final_sky*np.ones(n1),ls='-',c="k",label=r"Sky(λ) Em lines")#,lw=3,alpha=0.2)
            self.QE_curve     =  self.nax2_1.plot(self.new.QE_curve*np.ones(n1),ls='-.',c="k",label="Throughtput(λ)")#,lw=3,alpha=0.2)
            self.nax2_1.legend(loc='lower right',fontsize=8,title="Transmission curves")
            self.nax2_1.set_ylim((-0.05,1.05))


            # self.nax1bis = self.nax1.twinx()
            self.hw, self.hl =  self.Slitwidth/2/self.pixel_scale.value ,  self.Slitlength/2/self.pixel_scale.value
            self.nax.plot([n1/2 - self.hw,n1/2 + self.hw,n1/2 + self.hw,n1/2 - self.hw,n1/2 - self.hw],[n2/2 - self.hl,n2/2 - self.hl,n2/2 + self.hl,n2/2 + self.hl,n2/2 - self.hl],"-k")
            self.nax0.plot([n1/2 - self.hw,n1/2 + self.hw,n1/2 + self.hw,n1/2 - self.hw,n1/2 - self.hw],[n2/2 - self.hl,n2/2 - self.hl,n2/2 + self.hl,n2/2 + self.hl,n2/2 - self.hl],"-k", label="Slit=%0.1f'' × %0.1f''"%(self.Slitwidth,self.Slitlength))
            self.slit_text = AnchoredText("Slit=%0.1f'' × %0.1f''"%(self.Slitwidth,self.Slitlength), frameon=True, loc=4, pad=0.5,prop={'fontsize': 8})
            plt.setp(self.slit_text.patch, facecolor='white', alpha=0.5)
            self.nax.add_artist(self.slit_text)
            
            
            self.nax0.legend(loc='upper right',fontsize=8)

            self.profile = np.mean(im0.get_array().data[:,int(n1/2-self.Line_width.value/self.pixel_scale.value):int(n1/2+self.Line_width.value/self.pixel_scale.value)],axis=1)
            spatial_profile = self.im[:,:].mean(axis=1)
            # what means this convolution
            self.nax1.lines[0].set_ydata(spatial_profile)#np.convolve(spatial_profile,3,mode="same"))
            self.profile = np.mean(im0.get_array().data[:,:],axis=1)
            self.l1_s = self.nax1.plot(self.profile,label='Stack. spatial prof',c=self.l1[0].get_color())
            self.l2_s = self.nax1.plot(self.im_stack[int(n2/2 - self.PSF_source.value/self.pixel_scale.value/2/2.35):int(n2/2 + self.PSF_source.value/self.pixel_scale.value/2/2.35),:].mean(axis=0),label='Stack. spectral prof',c=self.l2[0].get_color())
            
            
            try:
                self.popt, self.pcov = curve_fit(gaus,np.arange(len(self.profile)),self.profile,p0=[np.ptp(self.profile), 50, 5, self.profile.min()])
            except RuntimeError:
                self.popt = [0,0,0,0]
            # self.fit = PlotFit1D(x= np.arange(len(self.profile)),y=self.profile,deg="gaus", plot_=False,ax=self.nax1bis,c="k",ls=":",P0=[np.ptp(self.profile), 50, 5, self.profile.min()])
            # self.nax1bis.plot( np.arange(len(self.profile)),gaus( np.arange(len(self.profile)),*self.popt),":k",label="SNR=%0.1f/%0.1f=%0.1f"%(self.popt[0]**2,self.profile[:20].std(),self.popt[0]**2/self.profile[:20].std()))
            self.l3_s = self.nax1.plot( np.arange(len(self.profile)),gaus( np.arange(len(self.profile)),*self.popt),":k",label="SNR=%0.1f/%0.1f=%0.1f"%(self.popt[0]**2,self.profile[:20].std(),self.popt[0]**2/self.profile[:20].std()))

            self.nax1.set_xlim((0,n1))
            self.nax1.legend(loc="upper right",fontsize=8,title="Averaged profiles")
            # self.nax1bis.legend(loc='upper right',fontsize=8,title="Averaged profiles")
            _,_,self.bars1 = self.nax2.hist(self.im.flatten(),bins=self.bins,log=True,alpha=0.3,color=self.l1[0].get_color(),label='Single image')
            _,_,self.bars2 = self.nax2.hist(self.im_stack.flatten(),bins=self.bins,log=True,alpha=0.3,color=self.l2[0].get_color(),label='Averaged stack')
            # TODO change the 40 in the next formula
            title = 'Signal kept=%i%%, RN kept=%i%%, Signal/tot=%i%%'%(100*self.new.Photon_fraction_kept[0], 100*self.new.RN_fraction_kept[0],100*(np.mean(self.im_stack[40:-40,:])-np.mean(self.im_stack[:20,:]))/np.mean(self.im_stack[40:-40,:]))
            self.nax2.plot([self.mod,self.mod],[0,100],c="k",ls=":",label="Bias %0.3f, PC limit %0.3f (%s):\n%s"%(self.mod,self.limit[i], self.new.counting_mode, title))
            self.nax2.plot([self.limit,self.limit],[0,100],c="k",ls=":")#,label="PC limit %i: %s"%(self.limit, title))

                        
            self.nax2.legend(loc='upper right',fontsize=8,title="Histogram")
            self.nax2.set_xlim(xmin=-10, xmax=np.nanmax(self.bins))
            self.cax = make_axes_locatable(self.nax).append_axes('bottom', size='15%', pad=0.05)
            self.cax0 = make_axes_locatable(self.nax0).append_axes('bottom', size='15%', pad=0.05)
            self.cbar1 = self.fig2.colorbar(im, cax=self.cax, orientation='horizontal')
            self.cbar2 = self.fig2.colorbar(im2, cax=self.cax0, orientation='horizontal')
            self.cbar1.formatter.set_powerlimits((0, 0))
            self.cbar2.formatter.set_powerlimits((0, 0))
            self.fig2.canvas.toolbar_position = 'bottom'
            self.fig2.tight_layout()
            plt.show(self.fig2)


        with self.out3:
            self.out3.clear_output(wait=True)
            self.current_cmap = cmocean.cm.deep# current_cmap = cmocean.cm.solar# self.current_cmap = cmocean.cm.thermal
            self.current_cmap.set_bad(color='black')
            n3 = int(np.sqrt(60*60*self.FOV_size)/self.Slitwidth)

            self.fig3 = plt.figure(figsize=(12, 8))
            gs = self.fig3.add_gridspec(2,2,height_ratios=[2,0.5])
            self.nax2s = self.fig3.add_subplot(gs[1,:])
            self.nax2s_secondary = self.nax2s.secondary_xaxis("top", functions=(f,g))


            self.nax20 = self.fig3.add_subplot(gs[0,0])
            self.nax21 = self.fig3.add_subplot(gs[0,1])

            self.nax20.set_xlabel("Spatial pixel")
            self.nax20.set_ylabel("Spatial pixel")
            self.nax21.set_xlabel("Spatial pixel")
            self.nax21.set_ylabel("Spatial pixel")
            self.nax20.set_title("Single cube: %ipix × %ipix"%(n2,n3))
            self.nax21.set_title("Stacked Cube: %0.1f' × %0.1f', full FOV= %0.1f' × %0.1f'"%(n2/self.pixel_scale.value/60,n3/self.pixel_scale.value/60,np.sqrt(self.FOV_size),np.sqrt(self.FOV_size)))


            self.nax2s.set_title("Wavelength (nm)",fontsize=10)
            self.nax2s.set_xlabel("Spectral pixel")
            self.nax2s.set_ylabel("Flux")

            self.ifs_cube = np.zeros((n2,n1,n3))
            # self.nax20.set_box_aspect(1)
            # self.nax21.set_box_aspect(1)
            self.wavelength_line1 = self.nax2s.axvline(int(n1/2)+0.5,ls='--',c='k')
            self.wavelength_line2 = self.nax2s.axvline(int(n1/2)+0.5,ls='--',c='k')
            if IFS:

                self.ifs_spectra = self.nax2s.plot(self.im_stack[int(n2/2),:],"k-",lw=3,alpha=0.1,label="Spectra: On source")
                self.ifs_spectra_stack = self.nax2s.plot(self.im[int(n2/2),:],"k--",alpha=0.5,label="Stacked spectra: On source")
            
                self.ifs_integ_spectra_stack = self.nax2s.plot(       np.nanmean(self.im[int(n2/2-self.Slitwidth/self.pixel_scale.value):int(n2/2+self.Slitwidth/self.pixel_scale.value),:],axis=0)      ,"k-",label="Integrated stacked spectra: On source")
                x1,x2 = n3/2-(n2/n2)*self.PSF_source.value/self.pixel_scale.value,n3/2+(n2/n2)*self.PSF_source.value/self.pixel_scale.value
                y1, y2 = n2/2-self.PSF_source.value/self.pixel_scale.value,n2/2+self.PSF_source.value/self.pixel_scale.value
                self.stack_square = self.nax20.plot([x1,x2,x2,x1,x1],[y2,y2,y1,y1,y2],"k:")
                
                self.ifs_spectra_background  = self.nax2s.plot(self.imaADU_without_source[int(n2/2),:],"k-",alpha=0)
                self.ifs_spectra_background_stack = self.nax2s.plot((self.im_stack-self.imaADU_stack_only_source)[int(n2/2),:],"k:",label="Spectra: Field edge (background)")


                gaussian = norm.pdf(np.arange(-n3,n3,2), loc=0, scale=self.PSF_source.value/2)
                ratios = (gaussian - gaussian.min()) / (gaussian.max() - gaussian.min())
                ratios_reshaped = ratios[np.newaxis, np.newaxis, :]
                # self.ifs_cube = self.im_stack[:, :, np.newaxis] * ratios_reshaped
                indices = np.array([np.random.permutation(n3) for _ in range(n1 * n2)])
                # Reshape indices to match the shape (n1, n2, n3)
                indices = indices.reshape(n2, n1, n3)                                


                self.ifs_cube = np.repeat(self.imaADU_without_source[:, :, np.newaxis], n3, axis=2)
                # random_indices = np.argsort(np.random.rand(n2,n1,n3), axis=1)
                self.ifs_cube = np.take_along_axis(self.ifs_cube, indices, axis=1)
                self.ifs_cube += self.imaADU_source[:, :, np.newaxis] * ratios_reshaped
                self.ifs_cube_stack = np.repeat( self.imaADU_stack_without_source[:, :, np.newaxis], n3, axis=2)
                self.ifs_cube_stack = np.take_along_axis(self.ifs_cube_stack, indices, axis=1)
                self.ifs_cube_stack += self.imaADU_stack_only_source[:, :, np.newaxis] * ratios_reshaped
            else:
                self.stack_square = self.nax20.plot([np.nan],[np.nan],"k:")
                self.ifs_slice = self.nax20.imshow(np.nan*self.ifs_cube[:,0,:], aspect="auto",cmap=self.current_cmap)
                self.ifs_slice_stack = self.nax21.imshow(np.nan*self.ifs_cube[:,0,:], aspect="auto",cmap=self.current_cmap)

                self.ifs_spectra = self.nax2s.plot([0,n1],[np.nan,np.nan],"k-",lw=3,alpha=0.1,label="Spectra: On source")
                self.ifs_spectra_stack = self.nax2s.plot([0,n1],[np.nan,np.nan],"k--",label="Stacked spectra: On source")
                self.ifs_integ_spectra_stack = self.nax2s.plot([0,n1],[np.nan,np.nan],"k-",label="Integrated stacked spectra: On source")
                self.ifs_spectra_background       = self.nax2s.plot([0,n1],[np.nan,np.nan],"k-",alpha=0)
                self.ifs_spectra_background_stack = self.nax2s.plot([0,n1],[np.nan,np.nan],"k:",label="Spectra: Field edge (background)")
                # self.ifs_slice = 0  # actually you need to initiate these variable in any case
            self.nax2s.legend(fontsize=7,loc="upper right")
            self.cax_slicer = make_axes_locatable(self.nax20).append_axes('right', size='5%', pad=0.05)
            self.cax_slicer0 = make_axes_locatable(self.nax21).append_axes('right', size='5%', pad=0.05)
            self.cbar_slicer1 = self.fig2.colorbar(im, cax=self.cax_slicer, orientation='vertical')
            self.cbar_slicer2 = self.fig2.colorbar(im2, cax=self.cax_slicer0, orientation='vertical')
            self.cbar_slicer1.formatter.set_powerlimits((0, 0))
            self.cbar_slicer2.formatter.set_powerlimits((0, 0))

            self.position1 = self.nax20.plot(int(n3/2),int(n2/2),"ro")
            self.position2 = self.nax21.plot(int(n3/2),int(n2/2),"ro")
            self.fig3.tight_layout()
            self.fig3.canvas.toolbar_position = 'bottom'
            plt.show(self.fig3)
            # if self.IFS.value:
            #     plt.show(self.fig3)
            # else:
            #     # self.fig3.clf()
            #     plt.close(self.fig3)
        display(HBox([self.output,new]))

    def hide_tab3(self):
        # Cacher l'onglet 3
        self.out3.layout.display = 'none'
        self.output_tabs.children = [self.out1, self.out2]  # Réinitialiser les onglets sans out3
        self.plot_shown = False  # Reset the plot flag when hiding tab 3    # self.output_tabs.selected_index = 0  # Sélectionner un autre onglet par défaut

    def show_tab3(self):
        # Afficher l'onglet 3
        self.out3.layout.display = 'block'
        self.output_tabs.children = [self.out1, self.out2, self.out3]  # Ajouter out3 à nouveau
        self.output_tabs.set_title(2, 'IFS Image')
        if not self.plot_shown:
            with self.out3:
                if self.first_plot:
                    self.fig3.canvas.draw()
                    self.first_plot = False
                else:
                    plt.show(self.fig3)
                # self.fig3.canvas.draw()
                # try:
                #     # Tenter de redessiner la figure sans plt.show() si elle existe
                #     self.fig3.canvas.draw_idle()
                # except:
                #     # Si cela échoue, cela signifie que la figure n'est plus active
                #     plt.show(self.fig3)
                self.plot_shown = True



    def on_instrument_change(self):
        # if ("FIREBall-2" in self.instrument.value):
        #     self.fb_tab.layout.display = 'block'
        #     self.controls.children = [ self.obs_tab, self.strat_tab,  self.inst_tab, self.spectro_tab, self.det_tab, self.im_tab, self.fb_tab]
        #     self.controls.set_title(6, 'FIREBall specific')
        # else:
        #     self.fb_tab.layout.display = 'none'
        #     self.controls.children = [ self.obs_tab, self.strat_tab,  self.inst_tab, self.spectro_tab, self.det_tab, self.im_tab]
        # self.on_tab_change(names='selected_index')

        self.change.value=True
        selected_index = self.output_tabs.selected_index
        if selected_index == 0:
            self.x_axis.layout.visibility = 'visible'  
            self.xlog.layout.visibility = 'visible'  
            self.ylog.layout.visibility = 'visible'  
            with self.out1:
                self.PSF_source.value-=0.01
            if ("FIREBall-2" in self.instrument.value):
                self.fb_tab.layout.display = 'block'
                self.controls.children = [ self.obs_tab, self.strat_tab,  self.inst_tab, self.spectro_tab, self.det_tab, self.fb_tab]
                self.controls.set_title(5, 'FIREBall specific')
            else:
                self.fb_tab.layout.display = 'none'
                self.controls.children = [ self.obs_tab, self.strat_tab,  self.inst_tab, self.spectro_tab, self.det_tab]                    
        else:
            self.x_axis.layout.visibility = 'hidden'  
            self.xlog.layout.visibility = 'hidden'  
            self.ylog.layout.visibility = 'hidden'  

            if ("FIREBall-2" in self.instrument.value):
                self.fb_tab.layout.display = 'block'
                self.controls.children = [ self.obs_tab, self.strat_tab,  self.inst_tab, self.spectro_tab, self.det_tab, self.im_tab, self.fb_tab]
                self.controls.set_title(5, 'Imaging')
                self.controls.set_title(6, 'FIREBall specific')
            else:
                self.fb_tab.layout.display = 'none'
                self.controls.children = [ self.obs_tab, self.strat_tab,  self.inst_tab, self.spectro_tab, self.det_tab, self.im_tab]                    
                self.controls.set_title(5, 'Imaging')
            if selected_index == 1:
                with self.out2:
                    self.PSF_source.value-=0.01
            elif selected_index == 2:
                with self.out3:
                    self.PSF_source.value+=0.01
        return


    def update(self,x_axis, counting_mode,Sky,acquisition_time,Signal,EM_gain,RN,CIC_charge,Dard_current,exposure,smearing,temperature,follow_temp,fwhm,QE,extra_background, log,xlog,
    Collecting_area, pixel_scale, Throughput, Spectral_resolution, SlitDims, dispersion,
    PSF_source,Line_width,wavelength,Δλ,Δx, Atmosphere, pixel_size,cosmic_ray_loss_per_sec,lambda_stack, change, 
    spectra,units,Throughput_FWHM, QElambda, atmlambda, fraction_lya,sky_lines, redshift, IFS
    ):
        """
        Update values in the ETC plot
        """
        self.show_tab3() if self.IFS.value else self.hide_tab3()

        with self.out1:
            if self.change.value:
                PSF_RMS_mask=fwhm[0]
                PSF_RMS_det=fwhm[1]
                # self.Slitwidth=SlitDims[0]
                # self.Slitlength=SlitDims[1]
                Slitwidth=SlitDims[0]
                Slitlength=SlitDims[1]
                self.Slitwidth=SlitDims[0]
                self.Slitlength=SlitDims[1]
                readout_time=exposure[0]
                exposure_time=exposure[1]
                if follow_temp:
                    self.follow_temp.value=follow_temp
                    self.Dard_current.value = self.dark_poly(temperature)
                    self.smearing.value = self.smearing_poly(temperature)
                    # self.CIC_charge.value = self.CIC_poly(self.temperature.value)
                if 1==1: # HACK here is just to be able to plot x_axis for fireball. Issue is that when change x_axis options, it comes back to the first choice
                    if "FIREBall" in self.instrument.value:
                        options = [x_axis] + self.fb_options if self.follow_temp.value else [x_axis] + self.fb_options_no_temp
                    else:
                        options =[x_axis] +  self.other_options
                    self.x_axis.options = options
                    if x_axis in options:
                        self.x_axis.value=x_axis
                    if  "OBSERVED SOURCE" in self.x_axis.value:
                        self.x_axis.value, x_axis = "Signal", "Signal"
                    elif  "OBSERVATION STRATEGY" in self.x_axis.value:
                        self.x_axis.value, x_axis = "Atmosphere","Atmosphere"
                    elif  "INSTRUMENT DESIGN" in self.x_axis.value:
                        self.x_axis.value, x_axis = "Collecting_area","Collecting_area"
                    elif  "SPECTROGRAPH DESIGN" in self.x_axis.value:
                        self.x_axis.value, x_axis = "Spectral_resolution","Spectral_resolution"
                    elif  "DECTECTOR PERFORMANCE" in self.x_axis.value:
                        self.x_axis.value, x_axis = "exposure_time", "exposure_time"
                    elif  "AMPLIFIED" in self.x_axis.value:
                        self.x_axis.value, x_axis = "EM_gain","EM_gain"



                args, _, _, locals_ = inspect.getargvalues(inspect.currentframe())
                if x_axis in locals_:
                    value = locals_[x_axis]
                else:
                    value = getattr(self,x_axis)
                    if (type(value) != float) & (type(value) != int):
                        value = instruments_dict[self.instrument.value][x_axis]
                    # error here! getattr(self,x_axis) can be 

                names = ["Signal","Dark current","Sky", "CIC", "Read noise","Extra Background"]
 

                self.smearing.layout.visibility = 'visible' if ("FIREBall-2" in self.instrument.value) & (self.counting_mode.value)    else 'hidden'
                self.temperature.layout.visibility = 'visible' if ("FIREBall-2" in self.instrument.value) &  (self.follow_temp.value)  else 'hidden'

                if x_axis == 'temperature':
                    temperature=np.linspace(self.temperature.min, self.temperature.max)
                    Dard_current = 10**self.dark_poly(temperature)
                    # smearing = np.poly1d([-0.0306087, -2.2226087])(temperature)
                    smearing = self.smearing_poly(temperature)
                # d = {name:np.linspace(rgetattr(self, '%s.min'%(name)), rgetattr(self, '%s.max'%(name))   name for self.fb_options_no_temp}

                self.len_xaxis = 50
                def space(a, b):
                    if (self.xlog.value) & (a>=0):
                        if a==0:
                            y = np.logspace(np.log10(np.max([a,0.0001])),np.log10(b),self.len_xaxis) 
                        else:
                            y = np.logspace(np.log10(a),np.log10(b),self.len_xaxis) 
    
                    else:
                        y = np.linspace(a,b,self.len_xaxis)
                    return y

                if self.output_tabs.get_state()["selected_index"]==self.output_tabs.children.index(self.out1):  

                    if x_axis == 'exposure_time':
                        exposure_time=space(1,self.time_max)
                    elif x_axis == 'Sky':
                        # Sky=np.logspace(-19,-15)
                        Sky = space(10**self.Sky.min,10**self.Sky.max)
                    elif x_axis == 'Signal':
                        # Signal=np.logspace(-19,-13)
                        Signal=space(10**self.Signal.min,10**self.Signal.max)
                    elif x_axis == 'EM_gain':
                        # EM_gain=np.linspace(self.EM_gain.min,self.EM_gain.max)
                        EM_gain=space(self.EM_gain.min,self.EM_gain.max)
                    elif x_axis == 'acquisition_time':
                        acquisition_time=space(self.acquisition_time.min,self.acquisition_time.max)
                    elif x_axis == 'RN':
                        RN=space(self.RN.min,self.RN.max)
                    elif x_axis == 'CIC_charge':
                        CIC_charge=space(0.001,self.CIC_charge.max)
                    elif x_axis == 'Dard_current':
                        Dard_current=space(self.Dard_current.min,self.Dard_current.max)
                    elif x_axis == 'readout_time':
                        readout_time=space(self.exposure.min,exposure_time)
                    elif x_axis == 'smearing':
                        smearing=space(self.smearing.min,self.smearing.max)
                    elif x_axis == 'temperature':
                        temperature=space(self.temperature.min,self.temperature.max)
                    elif x_axis == 'QE':
                        QE=space(self.QE.min,self.QE.max)
                    elif x_axis == 'PSF_RMS_mask':
                        PSF_RMS_mask=space(self.fwhm.min,self.fwhm.value[1])
                    elif x_axis == 'PSF_RMS_det':
                        PSF_RMS_det=space(self.fwhm.min,self.fwhm.max)
                    elif x_axis == 'extra_background':
                        extra_background=space(self.extra_background.min,self.extra_background.max)
                    elif x_axis == 'Δx':
                        Δx=space(self.Δx.min,self.Δx.max)#arcseconds
                    elif x_axis == 'Δλ':
                        Δλ=space(self.Δλ.min,self.Δλ.max)#angstrom
                    elif x_axis == 'Throughput':
                        Throughput=space(self.Throughput.min,self.Throughput.max)
                    elif x_axis == 'Atmosphere':
                        Atmosphere=space(self.Atmosphere.min,self.Atmosphere.max)
                    elif x_axis == 'Line_width':
                        Line_width=space(self.Line_width.min,self.Line_width.max)
                    elif x_axis == 'PSF_source':
                        PSF_source =  space(self.PSF_source.min,self.PSF_source.max)
                    elif x_axis == 'pixel_scale':
                        pixel_scale = space(self.pixel_scale.min,self.pixel_scale.max)
                    elif x_axis == 'pixel_size':
                        pixel_size = space(self.pixel_size.min,self.pixel_size.max)
                    elif x_axis == 'wavelength':
                        wavelength =space(self.wavelength.min,self.wavelength.max)
                    elif x_axis == 'Slitwidth':
                        Slitwidth =space(self.SlitDims.min,self.SlitDims.max)
                    elif x_axis == 'Slitlength':
                        Slitlength =space(self.SlitDims.min,self.SlitDims.max)
                    elif x_axis == 'Spectral_resolution':
                        Spectral_resolution = space(self.Spectral_resolution.min,self.Spectral_resolution.max)
                    elif x_axis == 'dispersion':
                        dispersion = space(self.dispersion.min,self.dispersion.max)
                    elif x_axis == 'lambda_stack':
                        # lambda_stack = np.linspace(np.log10(self.dispersion.value),np.log10(self.Bandwidth))
                        lambda_stack = space(self.dispersion.value,self.Bandwidth)
                    elif x_axis == 'Collecting_area':
                        Collecting_area = space(self.Collecting_area.min,self.Collecting_area.max)
                        # Collecting_area = np.linspace(0.01, 10)
                    elif x_axis == "cosmic_ray_loss_per_sec":
                        cosmic_ray_loss_per_sec=space(0,1/exposure[1])

                args, _, _, locals_ = inspect.getargvalues(inspect.currentframe())
                try:
                    new_value = locals_[x_axis]
                except KeyError:
                    new_value = getattr(self,x_axis)
                arg = np.argmin(abs(new_value - value))
                self.new = Observation(instrument=self.instrument.value,  exposure_time=exposure_time,Sky=Sky, acquisition_time=acquisition_time,counting_mode=counting_mode,Signal=Signal,EM_gain=EM_gain,RN=RN,CIC_charge=CIC_charge,Dard_current=Dard_current,readout_time=readout_time,smearing=smearing,extra_background=extra_background,i=arg,PSF_RMS_mask=PSF_RMS_mask,PSF_RMS_det=PSF_RMS_det,QE=QE,cosmic_ray_loss_per_sec=cosmic_ray_loss_per_sec, Throughput=Throughput, Atmosphere=Atmosphere,lambda_stack=lambda_stack,Slitwidth=Slitwidth, Bandwidth=self.Bandwidth,PSF_source=PSF_source,Collecting_area=Collecting_area,Δx=Δx,Δλ=Δλ,
                pixel_scale=pixel_scale, Spectral_resolution=Spectral_resolution,  dispersion=dispersion,Line_width=Line_width,wavelength=wavelength,  pixel_size=pixel_size,len_xaxis=self.len_xaxis, Slitlength=Slitlength,IFS=IFS)

                self.colors=self.new.colors

                if self.output_tabs.get_state()["selected_index"]==self.output_tabs.children.index(self.out1): 

                    try:
                        arg = np.argmin(abs(getattr(self.new,x_axis) - value))
                    except AttributeError:
                        arg= np.argmin(temperature - value)
                    try:
                        label = '%s [Best]=%s [%s]\nSNR [Best]=%0.2f, SNR=%0.2f'%(self.x_axis.value,float_to_latex(value),float_to_latex(new_value[np.nanargmax(self.new.SNR)]),self.new.SNR[arg],np.nanmax(self.new.SNR))
                    except (TypeError,ValueError) as e:
                        if ("FIREBall" in self.instrument.value) & (self.counting_mode.value):
                            label = '%s [Best]=%s [%s]\nSNR [Best]=%0.2f, SNR=%0.2f\nT=%0.1f sigma\nSignal kept=%i%%, RN kept=%i%%'%(self.x_axis.value,float_to_latex(value),float_to_latex(new_value[np.nanargmax(self.new.SNR)]),self.new.SNR[arg],np.nanmax(self.new.SNR),self.new.n_threshold[arg], 100*self.new.Photon_fraction_kept[arg], 100*self.new.RN_fraction_kept[arg])#, self.new.gain_thresholding[arg])
                        else:
                            label = '%s [Best]=%s [%s]\nSNR [Best]=%0.2f, SNR=%0.2f'%(self.x_axis.value,float_to_latex(value),float_to_latex(new_value[np.nanargmax(self.new.SNR)]),self.new.SNR[arg],np.nanmax(self.new.SNR))#, self.new.gain_thresholding[arg])

                    max_,min_=[],[]

                    for i,name in enumerate(self.new.names): 
                        self.ax0.lines[i].set_xdata(new_value)
                        self.ax0.lines[i].set_ydata(self.new.noises[:,i]/self.new.factor)
                        if self.new.percents[i,self.new.i] ==np.max(self.new.percents[:,self.new.i]):
                            self.ax0.lines[i].set_label(r"$\bf{➛%s}$: %0.2f (%0.1f%%)"%(name,self.new.noises[self.new.i,i]/self.new.factor[self.new.i],self.new.percents[i,self.new.i]))
                        else:
                            self.ax0.lines[i].set_label('%s: %0.2f (%0.1f%%)'%(name,self.new.noises[self.new.i,i]/self.new.factor[self.new.i],self.new.percents[i,self.new.i]))
                        max_.append(np.nanmax(self.new.noises[:,i]/self.new.factor))
                        min_.append(np.nanmin(self.new.noises[:,i]/self.new.factor))
                    self.ax0.lines[i+1].set_xdata(new_value)
                    self.ax0.lines[i+1].set_ydata(np.nansum(self.new.noises[:,:-1],axis=1)/self.new.factor)
                    self.ax0.lines[i+1].set_label('%s: %0.2f (%0.1f%%)'%("Total",np.nansum(self.new.noises[self.new.i,:-1])/self.new.factor[self.new.i],np.nansum(self.new.percents[:,self.new.i])  ))  #,"\nFlux [Best]=%0.2f, [%0.2f]",float_to_latex(self.new.signal_nsig_ergs[arg]),float_to_latex(float_to_latex(np.nanmin(self.new.signal_nsig_ergs)))))

                    self.ax0.legend(loc='upper right')
                    if x_axis in ["exposure_time","readout_time","PSF_RMS_mask","PSF_RMS_det","Slitwidth","Slitlength"]:
                        self.ax3.set_xlabel(x_axis.replace("_"," "))
                    else:
                        try:
                            self.ax3.set_xlabel(rgetattr(self, '%s.description_tooltip'%(x_axis)) )
                        except AttributeError:
                            self.ax3.set_xlabel(x_axis + "  [%s]"%(instruments["Unit"][instruments["Charact."]==x_axis][0]))

                    self.ax3.lines[0].set_data(new_value,  np.log10(self.new.extended_source_5s))
                    self.ax3.lines[0].set_label("SNR=5 Flux/Pow on one elem resolution (%0.2f-%0.2f)"%(np.log10(self.new.point_source_5s[arg]),np.nanmin(np.log10(self.new.point_source_5s))))


                    self.ax3.lines[1].set_data(new_value,  np.log10(self.new.extended_source_5s/np.sqrt(2)))

                    for v in self.v:
                        v.set_xdata([value,value])
                    self.v[-2].set_label(label)


                    for artist in self.ax2.collections+self.ax1.collections:
                        artist.remove()
                    self.stackplot2 = self.ax2.stackplot(new_value,self.new.snrs * np.array(self.new.noises).T[:-1,:]**2/self.new.Total_noise_final**2,alpha=0.7,colors=self.colors)
                    # labels =  ['%s: %0.3f (%0.1f%%)'%(name,getattr(self.new,"electrons_per_pix")[self.new.i,j],100*getattr(self.new,"electrons_per_pix")[self.new.i,j]/np.sum(getattr(self.new,'electrons_per_pix')[self.new.i,:])) for j,name in enumerate(self.new.names)]
                    contributions = getattr(self.new, "electrons_per_pix")[self.new.i, :]
                    percentages = 100 * contributions / np.sum(contributions)

                    # Find the index of the maximum contribution
                    max_index = np.argmax(percentages)

                    # Generate the labels, making the largest contribution bold
                    labels = [
                        r'%s: %0.3f (%0.1f%%)' % (name, contributions[j], percentages[j])
                        if j != max_index
                        else r"➛$\mathbf{%s}$: %0.3f (%0.1f%%)" % (name, contributions[j], percentages[j])
                        for j, name in enumerate(self.new.names)
                    ]                    
                    self.stackplot1 = self.ax1.stackplot(new_value,  np.array(self.new.electrons_per_pix).T,alpha=0.7,colors=self.colors,labels=labels)
                    self.ax1.legend(loc='upper right',title="Overall background: %0.3f (%0.1f%%)"%(np.nansum(self.new.electrons_per_pix[self.new.i,1:]),100*np.nansum(self.new.electrons_per_pix[self.new.i,1:])/np.nansum(self.new.electrons_per_pix[self.new.i,:])))
                    self.ax2.legend(loc='upper right')
                    self.ax3.legend(loc='upper right',fontsize=8)
                    self.ax2.set_xlim((np.max([np.min(new_value),1e-6]),np.max(new_value)))
                    self.ax2.set_xlim((np.min(new_value),np.max(new_value)))


                    if log:
                        self.ax0.set_yscale("log")
                        self.ax1.set_yscale("log")
                        self.ax2.set_yscale("log")
                        # self.ax0.set_ylim(ymin=np.nanmin(self.new.noises[:,:-1]/self.new.factor[:,None]),ymax=np.nanmax(np.nansum(self.new.noises[:,:-1],axis=1)/self.new.factor))
                        self.ax0.set_ylim(ymin=0,ymax=np.nanmax(np.nansum(self.new.noises[:,:-1],axis=1)/self.new.factor))
                        self.ax1.set_ylim(ymin=np.nanmin( np.array(self.new.electrons_per_pix[:,0])),ymax=np.max(np.sum(getattr(self.new,'electrons_per_pix'),axis=1)))
                        self.ax2.set_ylim(ymin=np.nanmin(np.array( self.new.snrs * np.array(self.new.noises).T[:-1,:]**2/self.new.Total_noise_final**2)[:,0]),ymax=np.nanmax(getattr(self.new,'SNR')))
                    else:
                        self.ax0.set_yscale("linear")
                        self.ax1.set_yscale("linear")
                        self.ax2.set_yscale("linear")
                        self.ax0.set_ylim((-0.1,np.nanmax(np.nansum(self.new.noises[:,:-1],axis=1)/self.new.factor)))
                        self.ax1.set_ylim((0,np.max(np.sum(getattr(self.new,'electrons_per_pix'),axis=1))))
                        self.ax2.set_ylim((0,np.nanmax(getattr(self.new,'SNR'))))
                    
                    if xlog:
                        try:
                            if (rgetattr(self,"%s.min"%(x_axis))<=0) & ( rgetattr(self,"%s.min"%(x_axis))  <  rgetattr(self,"%s.value"%(x_axis)) <  rgetattr(self,"%s.max"%(x_axis))):
                                self.ax0.set_xscale("symlog")
                            else:
                                self.ax0.set_xscale("log")
                        except AttributeError:
                            self.ax0.set_xscale("log")

                    else:
                        self.ax0.set_xscale("linear")

                    self.fig.canvas.draw()
                    # self.fig2.canvas.draw()

                else:
                    if "Spectra mNUV=" in x_axis:
                        Signal = float(spectra.split("=")[-1])
                        psf_source = 0.1
                        self.fraction_lya.layout.visibility = 'visible'
                    else: 
                        Signal = 20
                        psf_source = 4
                        self.fraction_lya.layout.visibility = 'hidden'
                    Sky = (self.new.Sky/exposure_time)#[arg]
                    flux = (self.new.Signal_el/exposure_time)#[arg]
                    self.im,self.im_stack, self.cube_stack, self.im0, source_im_wo_atm, self.imaADU_stack_only_source, self.imaADU_without_source, self.imaADU_stack_without_source, self.imaADU_source = self.new.SimulateFIREBallemCCDImage(Bias="Auto",  p_sCIC=0,  SmearExpDecrement=50000,  source=spectra,size=[n1, n2], OSregions=[0, max(n2,n1)], name="Auto", spectra="-", cube="-", n_registers=604, save=False, field="targets_F2.csv",QElambda=QElambda,atmlambda=atmlambda,fraction_lya= fraction_lya, Full_well=self.Full_well, conversion_gain=self.conversion_gain, Altitude=self.Altitude,Throughput_FWHM=self.Throughput_FWHM.value,sky_lines=self.sky_lines.value, redshift=self.redshift.value)

                    self.bins=np.linspace(np.nanmin(self.im),np.nanmax(self.im),100)

                    self.current_cmap.set_bad('red',1.)
                    if units=="ADU/frame": #ADU/frame: ok basic
                        factor=1
                    elif units=="amplified e-/frame": #e-/frame: divide by conversion gain and amplification gain
                        factor=1/self.conversion_gain
                    elif units=="e-/frame": #e-/frame: divide by conversion gain and amplification gain
                        factor=1/self.conversion_gain/EM_gain
                    elif units=="photons/frame": #photons/frame: account for QE
                        factor=1/self.conversion_gain/EM_gain/QE
                    elif units=="amplified e-/hour": #e-/hour: divide by exptime
                        factor=1/self.conversion_gain/exposure_time/3600
                    elif units=="e-/hour": #e-/hour: divide by exptime
                        factor=1/self.conversion_gain/EM_gain/exposure_time/3600
                    elif units=="photons/hour": #photons/hour: divide by exptime, account for QE
                        factor=1/self.conversion_gain/EM_gain/QE/exposure_time/3600
                        if "Spectra mNUV=" in x_axis:
                            Signal = float(spectra.split("=")[-1])
                            psf_source = 0.1
                            self.fraction_lya.layout.visibility = 'visible'
                        else: 
                            Signal = 20
                            psf_source = 4
                            self.fraction_lya.layout.visibility = 'hidden'

                self.f = lambda x: wavelength + (dispersion/10) * (x - n1/2)
                self.g = lambda x: (x - wavelength) / (dispersion/10) + n1/2

                if self.output_tabs.get_state()["selected_index"]==self.output_tabs.children.index(self.out2): 
                    with self.out2:
                        self.nax1_secondary.remove()
                        self.nax1_secondary = self.nax1.secondary_xaxis("top", functions=(self.f,self.g))
                        self.slit_text.txt.set_text("Slit=%0.1f'' × %0.1f''\n    = %0.2f × %0.2fkpc$^2$"%(self.Slitwidth,self.Slitlength,self.Slitwidth/cosmo.arcsec_per_kpc_proper(redshift+0.001).value,self.Slitlength/cosmo.arcsec_per_kpc_proper(redshift+0.001).value))

                        self.nax1.set_title('Wavelength (nm) - R = %i (λ/dλ) = %0.1fÅ = %0.1f km/s'%(Spectral_resolution, 10*wavelength/Spectral_resolution,299792.458/Spectral_resolution),fontsize=10)
                        if spectra =="Observed-frame: Baseline Spectra":
                            self.l2_s[0].set_label("Stack. spectral prof \nFWHM$_v$ = %0.1f km/s"%(Line_width * 299792.458/(10*wavelength )))
                        else:
                            self.l2_s[0].set_label("Stack. spectral prof")
                        # self.nax1.legend()
                        # self.source_spectra_sky[0].set_ydata(self.new.sky_im[40:-40,:].mean(axis=0)*np.ones(n1)   / self.new.sky_im[40:-40,:].mean(axis=0))
                        self.absorption[0].set_ydata(      self.new.atm_trans*np.ones(n1))# / np.max(self.new.atm_trans)                    
                        self.emission_lines[0].set_ydata(  (self.new.final_sky*np.ones(n1)   - np.min(self.new.final_sky)) / np.ptp(self.new.final_sky*np.ones(n1)   - np.min(self.new.final_sky))) 
                        self.QE_curve[0].set_ydata(  self.new.QE_curve*np.ones(n1) )#  / np.max(self.new.QE_curve)) 
                        # self.final_sky_before_convolution[0].set_ydata( (self.new.final_sky_before_convolution*np.ones(n1)   - np.min(self.new.final_sky_before_convolution)) / np.ptp(self.new.final_sky_before_convolution*np.ones(n1)   - np.min(self.new.final_sky_before_convolution))) 
                        # self.atm_trans_before_convolution[0].set_ydata(self.new.atm_trans_before_convolution*np.ones(n1)/ np.max(self.new.atm_trans_before_convolution))      
                        self.nax2_1_secondary.remove()
                        self.nax2_1_secondary = self.nax2_1.secondary_xaxis("top", functions=(self.f,self.g))

                        im = self.nax.imshow(self.im*factor, aspect="auto",cmap=self.current_cmap)
                        self.mod = mostFrequent(self.im_stack[:20,:].flatten())
                        # self.limit = self.mod+threshold*RN
                        self.limit = self.mod+self.new.n_threshold * RN

                        if counting_mode:
                            stacked_image = np.nansum(self.cube_stack>self.limit,axis=0)
                            im0 = self.nax0.imshow(stacked_image, aspect="auto",cmap=self.current_cmap)
                        else:
                            im0 = self.nax0.imshow(self.im_stack*factor, aspect="auto",cmap=self.current_cmap)
                        self.cbar1 = self.fig2.colorbar(im, cax=self.cax, orientation='horizontal')
                        self.cbar2 = self.fig2.colorbar(im0, cax=self.cax0, orientation='horizontal')
                        self.cbar1.formatter.set_powerlimits((0, 0))
                        self.cbar2.formatter.set_powerlimits((0, 0))

                        labels =  ['%s: %0.3f (%0.1f%%)'%(name,getattr(self.new,"electrons_per_pix")[self.new.i,j],100*getattr(self.new,"electrons_per_pix")[self.new.i,j]/np.sum(getattr(self.new,'electrons_per_pix')[self.new.i,:])) for j,name in enumerate(self.new.names)]


                        stacked_profile = np.mean(im0.get_array().data[:,int(n1/2 - Line_width/dispersion/2):int(n1/2 + Line_width/dispersion/2)],axis=1)
                        spatial_profile = self.im[:,int(n1/2 - Line_width/dispersion/2):int(n1/2 + Line_width/dispersion/2)].mean(axis=1)
                        self.nax1.lines[0].set_ydata(spatial_profile)

                        #TODO maybe remove the unit thing as it complexifies the magnification computation (maybe not a big deal as SNR is more important and it is invariant to units)
                        self.nax1.lines[1].set_ydata(self.im[int(n2/2 - PSF_source/pixel_scale/2):int(n2/2 + PSF_source/pixel_scale/2),:].mean(axis=0)) 
                        self.nax.lines[0].set_label("  \n".join(labels)) 
                        self.nax.legend(loc="upper left",handlelength=0, handletextpad=0, fancybox=True,markerscale=0,fontsize=8)
                
                        try:
                            self.popt, self.pcov = curve_fit(gaus,np.arange(len(stacked_profile)),stacked_profile,p0=[np.ptp(stacked_profile), 50, 5, stacked_profile.min()])
                        except RuntimeError:
                            self.popt = [0,0,0,0]

                        self.noise_res_element = self.im_stack[int(n2/2 - PSF_source/pixel_scale/2/2.35):int(n2/2 + PSF_source/pixel_scale/2/2.35),int(n1/2 - Line_width/dispersion/2/2.35):int(n1/2 + Line_width/dispersion/2/2.35)].std()/np.sqrt(self.new.resolution_element**2)
                        self.SNR = self.popt[0]**2 / self.noise_res_element
                        
                        self.Flux_ADU =  np.sum(gaus( np.arange(len(stacked_profile)),*self.popt)-self.popt[-1]) 
                        # self.Flux_ADU_counting =  np.sum(-np.log(1-( self.fit["function"]( np.arange(len(stacked_profile)),*self.fit["popt"])-self.fit["popt"][-1] )/(np.exp(-threshold*RN/EM_gain))))
                        self.e_s_pix = self.Flux_ADU * self.new.dispersion / exposure_time / self.new.N_images_true/self.conversion_gain  if counting_mode else  self.Flux_ADU * self.new.dispersion / EM_gain / exposure_time/self.conversion_gain
                        self.flux = self.e_s_pix / self.new.Throughput/ self.new.Atmosphere / QE / self.new.Collecting_area
                        photon_energy_erg = 9.93e-12
                        self.mag = -2.5*np.log10(self.flux*photon_energy_erg/(2.06*1E-16))+20.08
                        # Power 2 in SNR as it is in the definition of the gaussian fit.
                        # TODO we do no account for the poisson noise of the source, neither the fact that we use one resolution element

                        self.l1_s[0].set_ydata(stacked_profile) 
                        self.l2_s[0].set_ydata(np.mean(im0.get_array().data[int(n2/2 - PSF_source/pixel_scale/2):int(n2/2 + PSF_source/pixel_scale/2),:],axis=0))
                        self.l3_s[0].set_ydata(gaus( np.arange(len(stacked_profile)),*self.popt))
                        self.l3_s[0].set_label("SNR=%0.2f/%0.2f=%0.2f, mag=%0.1f"%(self.popt[0]**2,self.noise_res_element,self.SNR ,self.mag))


                        self.nax.set_title('Single image: FOV = %i" × %iÅ, Slit=%0.1f" × %0.1f"'%(100*self.pixel_scale.value,500*self.dispersion.value, self.Slitwidth,self.Slitlength)) 
                        self.nax0.set_title('Stacked image: Pixel size = %0.2f" × %0.2fÅ'%(self.pixel_scale.value,self.dispersion.value))

                        self.nax1.legend(loc="upper right",fontsize=8,title="Averaged profiles")

                        self.nax1.relim()
                        self.nax1.autoscale_view()

                        [b.remove() for b in self.bars2]
                        [b.remove() for b in self.bars1]
                        _,_,self.bars1 = self.nax2.hist(self.im.flatten(),bins=self.bins,log=True,alpha=0.3,color=self.l1[0].get_color(),label='Single image')
                        _,_,self.bars2 = self.nax2.hist(self.im_stack.flatten(),bins=self.bins,log=True,alpha=0.3,color=self.l2[0].get_color(),label='Averaged stack')
                        self.nax2.set_xlim(self.bins.min(),self.bins.max())


                        self.hw, self.hl = Slitwidth/2/self.pixel_scale.value ,  self.Slitlength/2/self.pixel_scale.value
                        if (np.round(self.Slitlength,3)!=np.round(Slitwidth,3)):
                            self.nax.lines[1].set_data([n1/2 - self.hw,n1/2 + self.hw,n1/2 + self.hw,n1/2 - self.hw,n1/2 - self.hw],[n2/2 - self.hl,n2/2 - self.hl,n2/2 + self.hl,n2/2 + self.hl,n2/2 - self.hl])
                            self.nax0.lines[0].set_data([n1/2 - self.hw,n1/2 + self.hw,n1/2 + self.hw,n1/2 - self.hw,n1/2 - self.hw],[n2/2 - self.hl,n2/2 - self.hl,n2/2 + self.hl,n2/2 + self.hl,n2/2 - self.hl])
                        else:
                            self.nax.lines[1].set_data(n1/2 + Slitwidth/2/self.pixel_scale.value * np.cos( np.linspace( 0 , 2 * np.pi , 50 ) ),n2/2+Slitwidth/2/self.pixel_scale.value * np.sin( np.linspace( 0 , 2 * np.pi , 50 ) ))
                            self.nax0.lines[0].set_data(n1/2 + Slitwidth/2/self.pixel_scale.value * np.cos( np.linspace( 0 , 2 * np.pi , 50 ) ),n2/2+Slitwidth/2/self.pixel_scale.value * np.sin( np.linspace( 0 , 2 * np.pi , 50 ) ))

                        # labels = "  \n".join(['%s: %0.3f (%0.1f%%)'%(name,getattr(self.new,"electrons_per_pix")[self.new.i,j],100*getattr(self.new,"electrons_per_pix")[self.new.i,j]/np.sum(getattr(self.new,'electrons_per_pix')[self.new.i,:])) for j,name in enumerate(self.new.names)])
                        # self.nax0.lines[0].set_label()
                        # self.nax0.legend(loc="upper left", handlelength=0, handletextpad=0, fancybox=True, markerscale=0, fontsize=8, title="Signal contribution")
                        contributions = getattr(self.new, "electrons_per_pix")[self.new.i, :]
                        max_index = np.argmax(100 * contributions / np.sum(contributions))
                        labels = "  \n".join([
                            r'%s: %0.3f (%0.1f%%)' % (name, contributions[j], 100 * contributions[j] / np.sum(contributions))
                            if j != max_index
                            else r"➛$\mathbf{%s}$: %0.3f (%0.1f%%)" % (name, contributions[j], 100 * contributions[j] / np.sum(contributions))
                            for j, name in enumerate(self.new.names)
                        ])

                        # Update the label and legend for the plot
                        self.nax0.lines[0].set_label(r'{}'.format(labels))
                        self.nax0.legend(loc="upper left", handlelength=0, handletextpad=0, fancybox=True, markerscale=0, fontsize=8, title="Signal contribution")
                        labels = [     r'%s: %0.2f (%0.1f%%)' % (name, self.new.noises[self.new.i, i] / self.new.factor[self.new.i], self.new.percents[i, self.new.i])     if self.new.percents[i, self.new.i] < np.max(self.new.percents[:, self.new.i])     else r"➛$\mathbf{%s}$: %0.2f (%0.1f%%)" % (name, self.new.noises[self.new.i, i] / self.new.factor[self.new.i], self.new.percents[i, self.new.i])     for i, name in enumerate(self.new.names)]
                        self.nax.lines[0].set_label(r'{}'.format("\n".join(labels)))
                        self.nax.legend(loc="upper left", handlelength=0, handletextpad=0, fancybox=True, markerscale=0, fontsize=8, title="%s Noise"%(self.instrument.value))

                        # self.nax0.legend(loc='upper right',fontsize=8)
                        self.nax2.lines[0].set_xdata([self.mod,self.mod])
                        self.nax2.lines[1].set_xdata([self.limit[arg],self.limit[arg]])
                        if "FIREBall" in  self.instrument.value:
                            try:
                                title = 'Signal kept=%i%%, RN kept=%i%%, Signal/tot=%i%%'%(100*self.new.Photon_fraction_kept[0], 100*self.new.RN_fraction_kept[0],100*(np.mean(self.im_stack[int(n2/2 - self.PSF_source.value/self.pixel_scale.value/2/2.35):int(n2/2 + self.PSF_source.value/self.pixel_scale.value/2/2.35),:])-np.mean(self.im_stack[:20,:]))/np.mean(self.im_stack[int(n2/2 - self.PSF_source.value/self.pixel_scale.value/2/2.35):int(n2/2 + self.PSF_source.value/self.pixel_scale.value/2/2.35),:]))
                            except IndexError as e:
                                title = 'Signal kept=%i%%, RN kept=%i%%, Signal/tot=%i%%'%(100*self.new.Photon_fraction_kept, 100*self.new.RN_fraction_kept[0],100*(np.mean(self.im_stack[int(n2/2 - self.PSF_source.value/self.pixel_scale.value/2/2.35):int(n2/2 + self.PSF_source.value/self.pixel_scale.value/2/2.35),:])-np.mean(self.im_stack[:20,:]))/np.mean(self.im_stack[int(n2/2 - self.PSF_source.value/self.pixel_scale.value/2/2.35):int(n2/2 + self.PSF_source.value/self.pixel_scale.value/2/2.35),:]))
                            self.nax2.lines[0].set_label("Bias %0.3f, PC limit %0.3f (%s):\n%s "%(self.mod,self.limit[arg], counting_mode, title))
                        else:
                            self.nax2.lines[0].set_label(" ")

                        # self.nax2.set_xlim(xmax=)       
                        self.nax2.legend(loc='upper right',fontsize=8,title="Histogram")
                        self.fig2.tight_layout()
                        self.fig2.canvas.draw()


                if len(self.output_tabs.children)>2: 
                    if self.output_tabs.get_state()["selected_index"]==self.output_tabs.children.index(self.out3): 
                        with self.out3:
                            if IFS:

                                n3 = int(np.sqrt(60*60*instruments_dict[self.instrument.value]["FOV_size"])/self.Slitwidth)
                                center = n1/2
                                self.f = lambda x: wavelength + (dispersion/10) * (x - center)
                                self.g = lambda x: (x - wavelength) / (dispersion/10) + center
                                self.nax2s_secondary.remove()
                                self.nax2s_secondary = self.nax2s.secondary_xaxis("top", functions=(self.f,self.g))

                                spectra = self.im[int(n2/2),:]#-Slitwidth/.
                                spectra_stacked = self.im_stack[int(n2/2),:]
                                background = self.im[int(n2/2),:]
                                background_stacked = self.im_stack[int(n2/2),:]
                                self.ifs_spectra[0].set_data(np.arange(len(self.im[int(n2/2),:])),self.im[int(n2/2),:]) 
                                self.ifs_spectra_stack[0].set_data(np.arange(len(self.im_stack[int(n2/2),:])),self.im_stack[int(n2/2),:])
                                self.ifs_spectra_background[0].set_data(np.arange(len(self.imaADU_without_source[int(n2/2),:])),self.imaADU_without_source[int(n2/2),:]) 
                                self.ifs_spectra_background_stack[0].set_data(np.arange(len(self.im_stack[int(n2/2),:])),(self.im_stack-self.imaADU_stack_only_source)[int(n2/2),:])
                                # TODO psf_mask or psf_det?
                                sigma = np.sqrt((PSF_RMS_mask/pixel_scale)**2+(self.PSF_source.value/pixel_scale)**2)
                                if self.Slitlength>Slitwidth:
                                    self.ifs_cube = np.zeros((n2,n1,n3))
                                    gaussian = np.exp(-np.linspace(-n2/2, n2/2, n3)**2 / (2 * sigma**2))    #norm.pdf(np.arange(-n3,n3,2), loc=0, scale=sigma)
                                    ratios = (gaussian - gaussian.min()) / (gaussian.max() - gaussian.min())
                                    ratios_reshaped = ratios[np.newaxis, np.newaxis, :]
                                    # Create an array of indices for the third dimension
                                    # Generate a 2D array where each row is a random permutation of the range n3
                                    indices = np.array([np.random.permutation(n3) for _ in range(n1 * n2)])
                                    # Reshape indices to match the shape (n1, n2, n3)
                                    indices = indices.reshape(n2, n1, n3)                                
                                    # Create the 3D cube by repeating the 2D background and indexing with the permuted indices
                                    self.ifs_cube = np.repeat(self.imaADU_without_source[:, :, np.newaxis], n3, axis=2)
                                    random_indices = np.argsort(np.random.rand(n2,n1,n3), axis=1)
                                    # Apply the permutation along the third axis using advanced indexing
                                    self.ifs_cube = np.take_along_axis(self.ifs_cube, indices, axis=1)
                                    # self.ifs_cube = np.repeat(self.imaADU_without_source[:, :, np.newaxis], n3, axis=2)
                                    # self.ifs_cube += self.im[:, :, np.newaxis] * ratios_reshaped
                                    
                                    self.ifs_cube += self.imaADU_source[:, :, np.newaxis] * ratios_reshaped
                                    self.ifs_cube_stack = np.repeat( self.imaADU_stack_without_source[:, :, np.newaxis], n3, axis=2)
                                    self.ifs_cube_stack = np.take_along_axis(self.ifs_cube_stack, indices, axis=1)
                                    # self.ifs_cube_stack += self.im_stack[:, :, np.newaxis] * ratios_reshaped
                                    self.ifs_cube_stack += self.imaADU_stack_only_source[:, :, np.newaxis] * ratios_reshaped
                                    self.position1[0].set_data([int(n3/2)],[int(n2/2)-Δx/pixel_scale]) 
                                    self.position2[0].set_data([int(n3/2)],[int(n2/2)-Δx/pixel_scale]) 

                                    self.nax20.set_title("%i pixels × %i spaxels"%(n2,n3))
                                    # self.nax21.set_title("%0.1f' × %0.1f', full FOV= %0.1f' × %0.1f'"%(n2/pixel_scale/60,n3/pixel_scale/60,np.sqrt(self.FOV_size),np.sqrt(self.FOV_size)))
                                    self.nax21.set_title("%0.1f' × %0.1f', full FOV= %0.1f' × %0.1f'"%(n2/pixel_scale/60,n2/pixel_scale/60,np.sqrt(self.FOV_size),np.sqrt(self.FOV_size)))
                                    self.nax20.set_ylabel("Real pixels")
                                    self.nax20.set_xlabel("Spatial pixels/slices (%0.1f'')"%(Slitwidth))
                                    self.nax21.set_xlabel("Spatial pixels/slices (%0.1f'')"%(Slitwidth))

                                else:
                                    self.nax20.set_xlabel("Spatial pixels/fibers (%0.1f''Ø)"%(Slitwidth))
                                    self.nax20.set_ylabel("Spatial pixels/fibers (%0.1f''Ø)"%(Slitwidth))
                                    self.nax21.set_xlabel("Spatial pixels/fibers (%0.1f''Ø)"%(Slitwidth))
                                    # self.nax21.set_ylabel("Spatial pixel (fiber)")
                                    spectra_source = np.nanmean(self.imaADU_source[int(n2/2-Slitwidth/pixel_scale):int(n2/2+Slitwidth/pixel_scale),:],axis=0)
                                    spectra_source_stack = np.nanmean(self.imaADU_stack_only_source[int(n2/2-Slitwidth/pixel_scale):int(n2/2+Slitwidth/pixel_scale),:],axis=0)
                                    background = np.nanmean(self.imaADU_without_source[int(n2/2-Slitwidth/pixel_scale):int(n2/2+Slitwidth/pixel_scale),:],axis=0)
                                    background_stack = np.nanmean(self.imaADU_stack_without_source[int(n2/2-Slitwidth/pixel_scale):int(n2/2+Slitwidth/pixel_scale),:],axis=0)
                                    # n2 = n3
                                    self.ifs_cube = np.zeros((n3,n1,n3)) 
                                    self.ifs_cube_stack = np.zeros((n3,n1,n3))
                                    #TODO here the spectra is constant!! so we can actually add some read noise (we will just increase the read noise in the spectral direction)
                                    # TODO how should we handle the poisson?
                                    # self.ifs_cube += np.random.poisson(self.imaADU_without_source[int(n2/2),:][np.newaxis,  :, np.newaxis])
                                    # self.ifs_cube_stack += np.random.poisson(self.imaADU_stack_without_source[int(n2/2),:][np.newaxis,  :, np.newaxis])
                                    self.ifs_cube += np.random.poisson(np.maximum(self.imaADU_without_source[int(n2/2),:],0)[np.newaxis,  :, np.newaxis]) + np.random.normal(0, RN, self.ifs_cube.shape) * self.conversion_gain
                                    self.ifs_cube_stack += np.random.poisson(np.maximum(self.imaADU_stack_without_source[int(n2/2),:],0)[np.newaxis,  :, np.newaxis]) + np.random.normal(0, self.RN.value/np.sqrt(int(self.new.N_images_true)), self.ifs_cube.shape) * self.conversion_gain
                                    ifs_cube = np.zeros((n3,n1,n3))
                                    # x = np.linspace(-n2/2, n2/2, n2)
                                    y = np.linspace(-n3/2, n3/2, n3)
                                    X, Y = np.meshgrid(y, y)
                                    gaussian = np.exp(-(X**2 + Y**2) / (2 * sigma**2))
                                    gaussian =  (gaussian - gaussian.min()) / (gaussian.max() - gaussian.min())
                                    self.nax20.set_title(r"%i × %i spaxels$^2$"%(n3,n3))
                                    self.nax21.set_title("%0.1f' × %0.1f', full FOV= %0.1f' × %0.1f'"%(n3/pixel_scale/60,n3/pixel_scale/60,np.sqrt(self.FOV_size),np.sqrt(self.FOV_size)))



                                    self.ifs_cube +=  np.random.poisson(np.maximum(spectra_source[np.newaxis,  :, np.newaxis] *   gaussian[:,  np.newaxis,:],0))
                                    self.ifs_cube_stack += np.random.poisson(np.maximum(spectra_source_stack[np.newaxis,  :, np.newaxis] *   gaussian[:,  np.newaxis,:],0))
                                    # TODO center it better
                                    self.position1[0].set_data([(n3)/2-0.5],[(n3)/2-Δx/pixel_scale]) 
                                    self.position2[0].set_data([(n3)/2-0.5],[(n3)/2-Δx/pixel_scale]) 
                                
                                # labels =  ['%s: %0.3f (%0.1f%%)'%(name,getattr(self.new,"electrons_per_pix")[self.new.i,j],100*getattr(self.new,"electrons_per_pix")[self.new.i,j]/np.sum(getattr(self.new,'electrons_per_pix')[self.new.i,:])) for j,name in enumerate(self.new.names)]
                                labels = [r"➛$\mathbf{%s}$: %0.3f (%0.1f%%)" % (name, getattr(self.new, "electrons_per_pix")[self.new.i, j], 100 * getattr(self.new, "electrons_per_pix")[self.new.i, j] / np.sum(getattr(self.new, "electrons_per_pix")[self.new.i, :])) 
                                        if getattr(self.new, "electrons_per_pix")[self.new.i, j] == np.max(getattr(self.new, "electrons_per_pix")[self.new.i, :]) 
                                        else '%s: %0.3f (%0.1f%%)' % (name, getattr(self.new, "electrons_per_pix")[self.new.i, j], 100 * getattr(self.new, "electrons_per_pix")[self.new.i, j] / np.sum(getattr(self.new, "electrons_per_pix")[self.new.i, :])) 
                                        for j, name in enumerate(self.new.names)]
                                self.position2[0].set_label("  \n".join(labels)) 
                                self.nax21.legend(loc="upper left",handlelength=0, handletextpad=0, fancybox=True,markerscale=0,fontsize=8,title="Signal contribution")#Overall background: %0.3f (%0.1f%%)"%(np.nansum(self.new.electrons_per_pix[self.new.i,1:]),100*np.nansum(self.new.electrons_per_pix[self.new.i,1:])/np.nansum(self.new.electrons_per_pix[self.new.i,:])))
                                

                                labels = [     r'%s: %0.2f (%0.1f%%)' % (name, self.new.noises[self.new.i, i] / self.new.factor[self.new.i], self.new.percents[i, self.new.i])     if self.new.percents[i, self.new.i] < np.max(self.new.percents[:, self.new.i])     else r"➛$\mathbf{%s}$: %0.2f (%0.1f%%)" % (name, self.new.noises[self.new.i, i] / self.new.factor[self.new.i], self.new.percents[i, self.new.i])     for i, name in enumerate(self.new.names)]
                                self.position1[0].set_label(r'{}'.format("\n".join(labels)))
                                self.nax20.legend(loc="upper left", handlelength=0, handletextpad=0, fancybox=True, markerscale=0, fontsize=8, title="%s Noise"%(self.instrument.value))


                                ifs_integ_spectra_stack =       np.nanmean(self.ifs_cube_stack[int(n2/2-Slitwidth/pixel_scale):int(n2/2+Slitwidth/pixel_scale),:,int(n3/2-(n3/n1)*Slitwidth/pixel_scale):int(n3/2+(n3/n1)*Slitwidth/pixel_scale)],axis=(0,2)) 
                                self.ifs_integ_spectra_stack[0].set_data(np.arange(len(self.im_stack[int(n2/2),:])),  ifs_integ_spectra_stack)
                                x1,x2 = (n3-1)/2-(n3/n2)*self.PSF_source.value/self.pixel_scale.value,(n3-1)/2+(n3/n2)*self.PSF_source.value/self.pixel_scale.value
                                y1, y2 = (n2-1)/2-self.PSF_source.value/self.pixel_scale.value,(n2-1)/2+self.PSF_source.value/self.pixel_scale.value
                                self.stack_square[0].set_data([x1,x2,x2,x1,x1],[y2,y2,y1,y1,y2])

                                self.ifs_slice       = self.nax20.imshow(np.nanmean(      self.ifs_cube[:,min(max(0,int(n1/2+0.5-lambda_stack/2)+int(Δλ/dispersion)),n1-1)   :min(max(0,int(n1/2+0.5+lambda_stack/2)+int(Δλ/dispersion)),n1-1),:],axis=1), aspect="auto",cmap=self.current_cmap)
                                self.ifs_slice_stack = self.nax21.imshow(np.nanmean(self.ifs_cube_stack[:,min(max(0,int(n1/2+0.5-lambda_stack/2)+int(Δλ/dispersion)),n1-1)   :min(max(0,int(n1/2+0.5+lambda_stack/2)+int(Δλ/dispersion)),n1-1),:],axis=1), aspect="auto",cmap=self.current_cmap)


                                self.cbar_slicer1 = self.fig3.colorbar(self.ifs_slice, cax=self.cax_slicer, orientation='vertical')
                                self.cbar_slicer2 = self.fig3.colorbar(self.ifs_slice_stack, cax=self.cax_slicer0, orientation='vertical')
                                self.cbar_slicer1.formatter.set_powerlimits((0, 0))
                                self.cbar_slicer2.formatter.set_powerlimits((0, 0))
                                
                                self.wavelength_line1.set_xdata([int(n1/2)+int(Δλ/dispersion)+0.5+(lambda_stack-1)/2,int(n1/2)+int(Δλ/dispersion)+0.5+(lambda_stack-1)/2])
                                self.wavelength_line2.set_xdata([int(n1/2)+int(Δλ/dispersion)+0.5-(lambda_stack-1)/2,int(n1/2)+int(Δλ/dispersion)+0.5-(lambda_stack-1)/2])
                                self.ifs_slice.set_clim(vmin=np.nanmin(self.ifs_cube), vmax=np.nanmax(self.ifs_cube))
                                # self.nax2s.set_ylim((np.nanmin(self.im[int(n2/2),:]),np.nanmax(self.im[int(n2/2),:])))
                                self.nax2s.set_ylim((np.nanmin(spectra_stacked),np.nanmax(spectra_stacked)))
                                self.nax2s.set_xlim((0,500))
                                self.fig3.tight_layout()
                                self.fig3.canvas.draw()
         
                    
    def update_instrument(self, instrument):
        with self.output:
            self.counting_mode.layout.visibility = 'visible' if ("FIREBall-2" in instrument) else 'hidden'
            self.follow_temp.layout.visibility = 'visible' if ("FIREBall-2" in instrument) else 'hidden'
            self.change.value=False
            keys = list(instruments_dict[instrument].keys())
            keys.remove("Signal")
            for key in keys:
                try:
                    rsetattr(self, '%s.value'%(key), instruments_dict[instrument][key]) 
                except AttributeError as e:
                    setattr( self, key, instruments_dict[instrument][key])
            self.IFS.value = True if self.dimensions==3 else False
            self.atmlambda.layout.visibility = 'visible' if (self.Altitude<100) else 'hidden'
            self.sky_lines.layout.visibility = 'visible' if (self.Altitude<10) else 'hidden'

            self.fwhm.value = (self.PSF_RMS_mask, self.PSF_RMS_det )
            self.SlitDims.value = (self.Slitwidth, self.Slitlength) 
            self.exposure.value = (self.readout_time, instruments[instrument][instruments["Charact."]=="exposure_time"][0] )
            self.smearing.layout.visibility = 'visible' if ("FIREBall-2" in instrument) & (self.counting_mode.value)    else 'hidden'
            self.temperature.layout.visibility = 'visible' if ("FIREBall-2" in instrument) &  (self.follow_temp.value)  else 'hidden'
            title = 'Instrument=%s, FOV=%samin$^2$, λ=%inm, Throughput=%i%%, Atm=%i%%, Platescale=%.1f, area=%0.1fm$^2$'%(instrument, instruments[instrument][instruments["Charact."]=="FOV_size"][0], instruments[instrument][instruments["Charact."]=="wavelength"][0], 100*instruments[instrument][instruments["Charact."]=="Throughput"][0], 100*instruments[instrument][instruments["Charact."]=="Atmosphere"][0], instruments[instrument][instruments["Charact."]=="pixel_scale"][0], instruments[instrument][instruments["Charact."]=="Collecting_area"][0])
            self.ax0.set_title(title,y=0.96,fontsize=10)
            self.change.value=True


            self.on_instrument_change()
            self.show_tab3() if self.IFS.value else self.hide_tab3()

            self.update(x_axis=x_axis, counting_mode=self.counting_mode.value,Sky=self.Sky.value,acquisition_time=self.acquisition_time.value,Signal=self.Signal.value,EM_gain=self.EM_gain.value,RN=self.RN.value,CIC_charge=self.CIC_charge.value,Dard_current=self.Dard_current.value,exposure=self.exposure.value,smearing=self.smearing.value,temperature=self.temperature.value,follow_temp=self.follow_temp.value,fwhm=self.fwhm.value,QE=self.QE.value,extra_background=self.extra_background.value,log=self.ylog.value,
            xlog=self.xlog.value , Collecting_area=self.Collecting_area.value , pixel_scale=self.pixel_scale.value , Throughput=self.Throughput.value , Spectral_resolution=self.Spectral_resolution.value , SlitDims=self.SlitDims.value , dispersion=self.dispersion.value , PSF_source=self.PSF_source.value , Line_width=self.Line_width.value , wavelength=self.wavelength.value , Δλ=self.Δλ.value , Δx=self.Δx.value , Atmosphere=self.Atmosphere.value , pixel_size=self.pixel_size.value , cosmic_ray_loss_per_sec=self.cosmic_ray_loss_per_sec.value, lambda_stack = self.lambda_stack.value,change=self.change,
            spectra=self.spectra.value,units=self.units.value,Throughput_FWHM=self.Throughput_FWHM.value, QElambda=self.QElambda.value, atmlambda=self.atmlambda.value, fraction_lya=self.fraction_lya.value, IFS=self.IFS.value)
            # print(self.Collecting_area.value)
        return

# TODO When slicer, we should not have an optimal with slit size, because the lost flux will go on the other slice............   
# TODO understand why SNR is shit for MUSE narrow... because it's done on a pixel size or element resolution (which is small!)
# TODO when lamda not good for wavelegnth try to show the issue and change the lambda limit of the sepctra
# HACK if fiber, be sure that self.Slitlength stays ==self.Slitwid

# [ ] add name to color bar, e- or ADU etc...,  add second yaxis in ", Solve QE(lambda) issue
# [x] noise correlated because of the way we exten slit array.
# [x] j'ajoute 2 fois le background with the current way!!!
# [x]  change resolution also for signal not only for atm
# [x], needed to add plt.show(fig3) : no image when going 2 times on slicer
# maybe aloow out3 even for slit (just saying that it would take n much more time)
# FB = ExposureTimeCalulator(instrument="TEST")
FB = ExposureTimeCalulator(instrument="FIREBall-2 2025")
# FB = ExposureTimeCalulator(instrument="FIREBall-1")
# FB = ExposureTimeCalulator()
# FB = ExposureTimeCalulator(instrument="SCWI")
# FB = ExposureTimeCalulator(instrument="ASPERA")
# instruments.write("/Users/Vincent/Github/fireball2-etc/notebooks/instruments.csv",overwrite=True)
# FB = ExposureTimeCalulator(instrument="KCWI blue")
# FB = ExposureTimeCalulator(instru/ment="Blue MUSE")
# FB = ExposureTimeCalulator(instrument="CETUS")
# FB = ExposureTimeCalulator(instrument="JUNO-UVS")
# FB = ExposureTimeCalulator(instrument="NISP")
# Hide flux as soon as spectra is selected

# TODO Ajouter la possibilité d'uploader un spectre... Mais du coup aussi un throughput



# be sure that SNR is right for IFS and slits: 1st plot=real SNR (integrating on several fibers/slicers if IFS), 2nd plot=SNR with only one fiber
# slitwidth and slit length should have different impact on FB1 (because one dimension impacts resolution and mixes with sky)
# clearly with IFS we don't cut flux by the slit base on the PSF at the slit (as it will end in the next slit) - except if the source is << slit?
# then we still integrate sky and flux based on slit size. so it is exactly the same impact as binning: 
# the smaller the fiber, the less we have flux for one fiber but the fllux per arcsec^2 on the image is the same!Is this currently true?

# IFS mode does not impact the SNR when slitdidth>>sigma source. But in the other case, we will loose some flux due to cutting
 #TODOOOOOOOO be sure to order well instrument dependancies lambda: sky, atmosphere, QE, 

In [2]:

# # FIREBall = ExposureTimeCalulator(instrument="FIREBall-2 2023")
# FIREBall.counting_mode
# from Observation import *
# Observation(exposure_time=np.linspace(50,1500,50),RN=50,smearing=0.2,counting_mode=True,plot_=False).compute_optimal_threshold(plot_=True,flux=0.03,size= (int(1e3),int(1e3)))

<!-- # Sky background
Sky background is given in line unit (LU= 1 ph/cm $^2$/s/sr).
To convert surface brightness $ergs/cm^2/s/asec^2$ Lto LU one can follow:
$$LU =   \frac{Flux[ergs/cm^2/s/asec^2]}{\frac{h c}{ \lambda} \times \frac{\pi}{ 180 \times 3600}^2 }   $$




The total sky background when stacking along the dispersion direction:
$$F[e-/pix/s] = Sky_{LU} \times Atm\times  Area_{telescope}[cm^2] \times Throughput  \times QE \times   Area_{pix} [Str] $$
The sky background per pixel:
$$F[e-/pix/s] = \frac{Sky_{LU} \times Size_{pix}[\AA] }{\lambda_{range}} \times Atm\times  Area_{telescope}[cm^2] \times Throughput  \times QE \times   Area_{pix} [Str] $$
For FIREBall, we have assumed a total sky background (extragalactic + atmospheric) of 500 continuum units [$ph /s/cm^2 /sr /\AA = LU/\AA$]
 -->


<!-- 
## Thresholding analysis
The threshold is computed by optimizing the SNR:

$$ SNR_1(T) = F_{True,pos}(T)\frac{S \times F_{e^->0,ADU>T}}{\sqrt{ F_{e^->0,ADU>T} \times(S + \sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2)  + F_{e^-=0,ADU>T} \frac{\sigma_{RN}^2}{Gain} }} 
\sim F_{True,pos}(T) \frac{S \sqrt{ F_{e^->0,ADU>T}}}{\sqrt{S + \sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2 }}$$



with:
- $\sigma_{RN}^2=\frac{RN}{G_{EM}} \times \frac{\sum RN[RN>T]}{\sum RN} = \frac{RN}{EMgain}  \times \%pix_{e^-=0}^{ADU>T}$: ie 60e-/2000 x the fraction of pixels above the threshold which dit not receive any photo-electrons
- $ F_{e^->0,ADU>T}$ :  fraction of pixels above the threshold which received more than 0 electons
- $ F_{e^-=0,ADU>T}$ :  fraction of pixels above the threshold which did not receive any electron
- $F_{True,pos} = \frac{F_{e^->0,ADU>T}}{F_{ADU>T}} $ : the fraction of true positives = the fraction of pixels above the threshold which received more than 0 electons. This uncertainty term is not in the final SNR formula as it converges towards ~1 after stacking as the average pixel value will converge towards the real number of incoming photo-electrons


The thresholding depends on 5 factors:
- **The gain**: the higher the gain the easier it is to differentiate pixels who received a photo-electron and those which did not
- **The read noise**: the lower the RN the easier it is to differentiate pixels who received a photo-electron and those which did not
- **The low CTE** smears the charges, from pixels who did receive a photo-electron to pixel who did not and lower the effective gain $G_{EM,eff}$
- **The signal**: because thresholding only allows to recover 0 and 1, thresholding is only efficient at flux <<1e-/exp 
- **The additional noise**: $\sigma = \sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2$ has relatively small impact on the impact of the thresholding

**At the end, the thresholding impact is:**
- Highly non linear, many things to learn
- The efficiency of the thresholding depends mostly of the ratio $\frac{G_{EM,eff}}{RN}$, which extremely impacted by the smearing.
- The thresholding SNR gain ($SNR_{threshold}$ /$SNR_{analogic}$) lies within $[0,\sqrt{2}]$. Condditions are pretty stricts to get som thresholding efficiency and smearing makes the upper bound falls dramatically. 
- Therefore if might not worth it to loose SNR in order to be in these very conditions as we will loose it anyway in the final budget.
- The thresholding SNR gain seems less impacted by read noise. Which means that if we manage to do some (linear?) inversion of the smearing which increases the noise, it might still increase the thresholding gain above 1. 
- The only real game changer in terms of SNR is the smearing 
- **Fraction kept (photo-electrons above threshold)**
    - Important dependance on smearing and flux, less important one on gain and read noise.
    - At low smearing the bigger the ratio gain/ron the higher the fraction of charges kept.
    - Evolution with smearing is dramatic, higher gain does not help much, lower RN helps
    - Fraction increase with flux 
- **Fraction fake detections (0e- pixels above threshold)**
    - Increases importantly with smearing
    - does not depend much on read noise (small positive correlation)
    - decreases with gain
    - Increases as Flux increase. Not sure to understand why... Maybe because the number of pixels with 0 photo-e- cecreases 
- **Thresholding SNR gain**
    - Thresholding is efficient only if >1
    - Dramatically decreases with smearing (No interest of thrsholding above smearing legth of 0.6)
    - Increases with gain and decreases with read noise if no smearing
    - if smearing, the read noise has no impact.
    - Increases with flux
- **Threshold**
    - increase extremely with smearing
    - Without smearing, poor dependance on gain and readnoise
    - If smearing, increases as readnoise decreases
    - If smearing: Increasing with gain!
    - Increases with flux




|   **Thresholding dependancy table**                   | **Smearing <br />  ( 0 - 2 )** | **Read noise  <br /> ( 30 - 120 )** | **Gain  <br /> ( 800 - 2500 )** | **Flux <br /> ( 0.01 - 1.5 )** |
|:--------------------:|:------------------:|:-----------------------:|:-------------------:|:------------------:|
|     **Photo-electron Fraction kept**    |    --- <br /> 92% - 66%    |       -  <br /> 72% - 71%       |     .   <br /> 72% - 71%    |     +  <br /> 75% - 76%    |
| **RN fraction kept** |  +++  <br /> 0.6% - 12.4%  |       +  <br /> 0.7% - 1%       |    -  <br /> 1.1% - 0.5%    |    ++?  <br /> 0.3% - 3%   |
|     **Threshold**    |   +++  <br /> 3 σ - 11 σ   |      --  <br /> 18 σ - 5 σ      |     + <br />  5 σ - 16 σ    |    +? <br />  8 σ - 13 σ    |
|        **Thresholding SNR gain**       |     --- <br />  1.3 - 0.6   |        --  <br /> 1.35 - 1.22        |      +  <br /> 1.13 - 1.18      |      ++  <br /> 1.16 - 1.21     |
 -->


<!-- # <center>FIREBall-2 SNR calculator </center>

### Explanation
- Gives the estimated signal-to-noise ratio for an extended source on a resolution element
- **Variables:** Source's flux, sky, aquisition time, exposure time, EM gain, dark current, CIC, readnoise, readout time, thresholding, smearing exponential length and temperature if you check it (based on a first rough evolution of smearing and dark current with temperature, therefore changing the temperature will change smearing and dark accordingly.)
- **Fixed Values:** QE = 45%, resolution element=57 mu, pixel_size = 13 mu, throughput = 13%, atmosphere transmission = 50%
- **Outputs:** In the top pannel, the noise of the different sources (Signal, Dark, Sky, CIC, RN) is given in e- per resolution element per N frames on the total acquisition time. The middle pannel we give the average e-/pix value for each component (before stacking). We give the relative fraction of all sources of noise and the resulting SNR. When thresholding is applied, the smearing can be changed as it will impact the position of the threshold that optimizes the SNR. For this optimal threshold, the fraction of signal and readnoise kept (above the threshold) is given.
- **Caution:**  it appears that for EMCCDs the dependencies are pretty complex/non-linear. In particular, the lack of models on the impact of amplification, thresholding, and register clearing... combined with smearing on the final SNR makes risky the fact of relying only on modeling to choose the optimal temperature/gain/etc. That's why I suggest an [end-to-end test](https://docs.google.com/document/d/1SpiEK8MBmYduKUFvWmnTOfkfsX4VWs6JMF5-YJKDELo/edit?usp=sharing) validating this: 


### Analysis
- **Smearing:** At -120C, the low device's CTE smears the pixels' counts and then lowers the effective gain. This lowering factor is huge (>2) for a smearing length of 1.5, but is much smaller for a smearing length of 0.7 (~1.2). 
It seems that we already had a comparabale 0.7 smearing length with previous controller at 10MHz (tests February 7th and 8th 2019) at same EMgain (1600) and temperature (-95C). Which might mean that the 1MHz reading does not decrease significantly the smearing. **With the current (2022) smearing length, thresholding does not increase the SNR!! If we manage to invert the smearing, even if it doubles the readnoise, we could recover most of the thresholding efficiency.**
- **Photon-counting:** The thresholding effect can be misleading as it depends on Flux, EMgain, RN, smearing and noise($\sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2$). But actually it depends mostly on the ratio $\frac{G_{EM,eff}}{RN}$. For a ratio >> 10 the thresholding is efficient. As stated above for a smearing length of 0.7, EMgain and effective_gain are close. A consequence of that, is that at the EMgains we use ($\sim1600$) and smearing $\sim 0.7$, the thresholding is almost as efficient with RN=50 and RN=107 as the ratio $\frac{G_{EM,eff}}{RN}$ stays >>10. This was not the case in 2018 when the smearing length was 1.5. This is why in photon counting mode, you will see a very small impact of read noise change.
- **Reading frequency tradeoff:** Based on above, the change from 10MHz to 1MHz as minor consequences on the total SNR.
    - In photon counting mode and with low smearing, the decrease of the RN has quasi no effect (~1%) on the SNR
    - The transition from 1.4 to 3sec readout would mean a loss of SNR of ~2%, transition to 10sec about ~10% 
    - Therefore the real critical point is the possible need to add or not a shutter above the tank with the 1MHz controller as it will complexify guidance
- **Temperature Tradeoff:** Conclusion depends on the definition of SNR on thresholded images (see below): In both cases, the SNR decreases with smearing but in one case smearing has more important impact and compensate  dark current giving rise to an optimal temperature. Though, the fact that we will never get rid of dark current while we could actually manage to inverse the smearing could suggest staying at low temperature (-120 to -110).

 
  
  
|             **Parameters**             |          **2022 1Mhz controller**           | **Values 2018** |
| :------------------------------------: | :-----------------------------------------: | :-------------: |
|          **Temperature (C)**           |                     -95                     |      -115       |
|       **Conversion gain ADU/e-**       |                    1/4.5                    |      0.53       |
|           **Read noise e-**            |            50 -> 60 longer cable            |       107       |
|         **pCIC e− /pix/frame**         |                    0.003                    |      0.004      |
|         **sCIC e− /pix/frame**         |                   ~0.002                    |     ~0.002      |
|  **semi-amplified CIC e− /pix/frame**  |                    ~0.02                    |     ~0.03-4     |
|           **EM gain  e-/e-**           |          O                    2000          |      1400       |
|   **Smearing exponent. length pix**    |              L~0.6 ± 0.1  pix               |   L~1.5 ± 0.1   |
|       **Dark current e− /pix/h**       |                   ~2±0.5                    |   0.5 ± 0.03    |
| **Cosmetics % pixels bottom detector** |                     ~2%                     |   <<1% (TBD)    |
|         **Exposure time sec**          |                     50                      |       50        |
|          **Readout time sec**          | ~10 (all), ~3(physical) +2 better clearance |      ~1.4       |


<!-- 
### Need to add
- Predictions for point source, sources on several resolution element, stack of galaxies
 -->

<!-- Based on the 2019 and 2022 images the consequences of a 20 degrees increase is important. With the first approximations, the increase of the dark current combined with the decrease of the smearing length might lead to a 5% SNR increase in counting mode (+sharper PSF). Though, for now, the major consequence of the temperature increase seems to be the high fraction of cosmetics on the lower part of the image. A significant fraction could be corrected for with appropriated software but the fraction can become extremely high and prevent good correction (needs further implementation). This is not taken into account in this model. Decreasing the tempetrature to -100C (TBD) might be a good trade off (in 2019 we had 0.8 smearing at -95C and EMgain=1600). For very bright objects the shot noise becomes dominant and then the impact of dark current is smaller so the SNR peaks at higher temps (80).
 --> 

<!-- 
## Thresholding analysis
The threshold is computed by optimizing the SNR:

$$ SNR_1(T) = F_{True,pos}(T)\frac{S \times F_{e^->0,ADU>T}}{\sqrt{ F_{e^->0,ADU>T} \times(S + \sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2)  + F_{e^-=0,ADU>T} \frac{\sigma_{RN}^2}{Gain} }} 
\sim F_{True,pos}(T) \frac{S \sqrt{ F_{e^->0,ADU>T}}}{\sqrt{S + \sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2 }}$$



with:
- $\sigma_{RN}^2=\frac{RN}{G_{EM}} \times \frac{\sum RN[RN>T]}{\sum RN} = \frac{RN}{EMgain}  \times \%pix_{e^-=0}^{ADU>T}$: ie 60e-/2000 x the fraction of pixels above the threshold which dit not receive any photo-electrons 
- $ F_{e^->0,ADU>T}$ :  fraction of pixels above the threshold which received more than 0 electons
- $ F_{e^-=0,ADU>T}$ :  fraction of pixels above the threshold which did not receive any electron
- $F_{True,pos} = \frac{F_{e^->0,ADU>T}}{F_{ADU>T}} $ : the fraction of true positives = the fraction of pixels above the threshold which received more than 0 electons. This uncertainty term is not in the final SNR formula as it converges towards ~1 after stacking as the average pixel value will converge towards the real number of incoming photo-electrons

The thresholding depends on 5 factors:
- **The gain**: the higher the gain the easier it is to differentiate pixels who received a photo-electron and those which did not
- **The read noise**: the lower the RN the easier it is to differentiate pixels who received a photo-electron and those which did not
- **The low CTE** smears the charges, from pixels who did receive a photo-electron to pixel who did not and lower the effective gain $G_{EM,eff}$
- **The signal**: because thresholding only allows to recover 0 and 1, thresholding is only efficient at flux <<1e-/exp 
- **The additional noise**: $\sigma = \sigma_{CIC}^2 + \sigma_{DC}^2+ \sigma_{SKY}^2$ has relatively small impact on the impact of the thresholding

**At the end, the thresholding impact is:**
- Highly non linear, many things to learn
- The efficiency of the thresholding depends mostly of the ratio $\frac{G_{EM,eff}}{RN}$, which extremely impacted by the smearing.
- The thresholding SNR gain (SNR$_{threshold}$/SNR$_{analogic}$) lies within $[0,\sqrt{2}]$. Condditions are pretty stricts to get som thresholding efficiency and smearing makes the upper bound falls dramatically. 
- Therefore if might not worth it to loose SNR in order to be in these very conditions as we will loose it anyway in the final budget.
- The thresholding SNR gain seems less impacted by read noise. Which means that if we manage to do some (linear?) inversion of the smearing which increases the noise, it might still increase the thresholding gain above 1. 
- The only real game changer in terms of SNR is the smearing 
- **Fraction kept (photo-electrons above threshold)**
    - Important dependance on smearing and flux, less important one on gain and read noise.
    - At low smearing the bigger the ratio gain/ron the higher the fraction of charges kept.
    - Evolution with smearing is dramatic, higher gain does not help much, lower RN helps
    - Fraction increase with flux 
- **Fraction fake detections (0e- pixels above threshold)**
    - Increases importantly with smearing
    - does not depend much on read noise (small positive correlation)
    - decreases with gain
    - Increases as Flux increase. Not sure to understand why... Maybe because the number of pixels with 0 photo-e- cecreases 
- **Thresholding SNR gain**
    - Thresholding is efficient only if >1
    - Dramatically decreases with smearing (No interest of thrsholding above smearing legth of 0.6)
    - Increases with gain and decreases with read noise if no smearing
    - if smearing, the read noise has no impact.
    - Increases with flux
- **Threshold**
    - increase extremely with smearing
    - Without smearing, poor dependance on gain and readnoise
    - If smearing, increases as readnoise decreases
    - If smearing: Increasing with gain!
    - Increases with flux

 -->

<!-- 

|   **Thresholding dependancy table**                   | **Smearing <br />  ( 0 - 2 )** | **Read noise  <br /> ( 30 - 120 )** | **Gain  <br /> ( 800 - 2500 )** | **Flux <br /> ( 0.01 - 1.5 )** |
|:--------------------:|:------------------:|:-----------------------:|:-------------------:|:------------------:|
|     **Photo-electron Fraction kept**    |    --- <br /> 92% - 66%    |       -  <br /> 72% - 71%       |     .   <br /> 72% - 71%    |     +  <br /> 75% - 76%    |
| **RN fraction kept** |  +++  <br /> 0.6% - 12.4%  |       +  <br /> 0.7% - 1%       |    -  <br /> 1.1% - 0.5%    |    ++?  <br /> 0.3% - 3%   |
|     **Threshold**    |   +++  <br /> 3 σ - 11 σ   |      --  <br /> 18 σ - 5 σ      |     + <br />  5 σ - 16 σ    |    +? <br />  8 σ - 13 σ    |
|        **Thresholding SNR gain**       |     --- <br />  1.3 - 0.6   |        --  <br /> 1.35 - 1.22        |      +  <br /> 1.13 - 1.18      |      ++  <br /> 1.16 - 1.21     |
 -->
